###### Assignment 2 - Interplanetary Transfer

For this assignment, you will use a Lambert targeter to generate a first guess for an unperturbed interplanetary transfer. Subsequently, you will use various numerical propagation models that include perturbations to analyze the trajectory in more detail, and compute trajectory corrections such that your trajectory meets its boundary conditions (departure and arrival) in the perturbed environment,

In this assignment, you are required to modify this Jupyter notebook. Unlike assignment 1, this assignment is in a single notebook, with the code for all questions and the assignment description all merged into a single document.


**General Instructions**

In this assignment, you will use a so-called Lambert targeter (a tool that solves Lambert's problem) to generate an initial guess for an interplanetary direct high-thrust transfer. The Lambert targeter takes as input:

<ul>
    <li>A departure position $\mathbf{r}_0$</li>
    <li>An arrival position $\mathbf{r}_E$</li>
    <li>A time of flight $T$</li>
    <li>A central body gravitational parameter $\mu$</li>
</ul>


The Lambert targeter then generates the Keplerian trajectory between $\mathbf{r}_{0}$ and $\mathbf{r}_{E}$, with the given time of flight $T$. Since the initial and final positions uniquely define the trajectory (assuming a prograde single-revolution trajectory; ignoring rare singular cases), the full state along this Keplerian trajectory (or Lambert arc) are *outputs* of the Lambert targeter. We denote the Cartesian state function of this Lambert arc as $\bar{\mathbf{x}}(t)$. For your situation, the initial and final position $\mathbf{r}_{0}$ and $\mathbf{r}_{E}$ of the full trajectory are the positions of the center of mass of Earth and Mars or Venus (w.r.t. the Sun), at times $t_{0}$ and $t_{E}$, respectively (with values depending on your student number, to be found in the $\texttt{assignment2Input-2020-2021.txt}$ file on Brightspace under Assignment 2).

All analysis on the output data can be done in the notebook. However, if you would like to use a different piece of software (*e.g.* Matlab) for your analyses, the relevant data is provided as output to data files, in a number of manners:

* By calling the `propagate_trajectory` function, the propagated state of the spacecraft and the associated dependent variables will be saved to a file, as well as the state of the Lambert arc $\bar{\mathbf{x}}$ at the epochs of the numerical integration. See the in-code comments of the `write_propagation_results_to_file` function in the helper functions block code file for more details.
* For question 3, the `write_propagation_results_to_file` function is called directly from the $\texttt{main}$ function.


**Before starting the assignment, read the submission instructions given at the end of this notebook.**




In [ ]:
''' 
Copyright (c) 2010-2020, Delft University of Technology
All rigths reserved

This file is part of the Tudat. Redistribution and use in source and 
binary forms, with or without modification, are permitted exclusively
under the terms of the Modified BSD license. You should have received
a copy of the license with this file. If not, please or visit:
http://tudat.tudelft.nl/LICENSE.
'''
import numpy as np
from tudatpy import elements
from tudatpy.io import save2txt
from tudatpy.kernel import constants
from tudatpy.kernel.interface import spice_interface
from tudatpy.kernel.simulation import environment_setup
from tudatpy.kernel.simulation import estimation_setup
from tudatpy.kernel.simulation import propagation_setup
from tudatpy.kernel.astro import two_body_dynamics
from tudatpy.kernel.astro import conversion
from matplotlib import pyplot as plt

# STUDENT CODE TASK (fill in 4456521	4	2056.746011	215.4577652+
departure_epoch = 2056.746011 * constants.JULIAN_DAY
time_of_flight = 215.4577652 * constants.JULIAN_DAY
arrival_epoch = departure_epoch + time_of_flight
target_body = "Mars"

# Global settings
fixed_step_size = 3600.0
global_frame_origin = "SSB"
global_frame_orientation = "ECLIPJ2000"

# Helper variables for question 4
current_question = 0;
rsw_acceleration_magnitude = [0,0,0]

# Load spice kernels.
spice_interface.load_standard_kernels()

# Set output directory
output_directory = "./SimulationOutput/"

# Helper Functions (DO NOT MODIFY)

In [ ]:
def write_propagation_results_to_file( dynamics_simulator, lambert_arc_state_model, file_output_identifier, output_directory):

    """
    This function will write the results of a numerical propagation, as well as the Lambert arc states at the epochs of the
    numerical state history, to a set of files. Two files are always written when calling this function (numerical state history, a
    and Lambert arc state history). If any dependent variables are saved during the propagation, those are also saved to a file
    
    Parameters
    ----------
    dynamics_simulator : Object that was used to propagate the dynamics, and which contains the numerical state and dependent
                         variable results
       
    lambert_arc_state_model : Lambert arc state model as returned by the get_lambert_problem_result() function
    
    file_output_identifier : Name that will be used to correctly save the output data files
    
    output_directory : Directory to which the files will be written
    
    Files written
    -------------
    
    <output_directory/file_output_identifier>_numerical_states.dat
    <output_directory/file_output_identifier>_dependent_variables.dat
    <output_directory/file_output_identifier>_lambert_statess.dat

    
    Return
    ------
    None
    
    """
        
    # Save numerical states
    simulation_result = dynamics_simulator.state_history
    save2txt(solution= simulation_result, filename=output_directory + file_output_identifier + "_numerical_states.dat", directory="./", column_names=None )
   
    # Save dependent variables
    dependent_variables = dynamics_simulator.dependent_variable_history
    if len(dependent_variables.keys()) > 0:
        save2txt(solution= dependent_variables, filename=output_directory + file_output_identifier + "_dependent_variables.dat", directory="./", column_names=None )
   
    # Save Lambert arc states
    lambert_arc_states = get_lambert_arc_history( lambert_arc_state_model, simulation_result )
    
    save2txt(solution= lambert_arc_states, filename= output_directory + file_output_identifier + "_lambert_states.dat",  directory="./", column_names=None )
    
    return

def get_lambert_problem_result(bodies, target_body, departure_epoch, arrival_epoch):
    
    # Gravitational parameter of the Sun
    central_body_gravitational_parameter = bodies.get_body( "Sun" ).gravitational_parameter
    
    # Set initial and final positions for Lambert targeter
    initial_state = spice_interface.get_body_cartesian_state_at_epoch(
       target_body_name="Earth",
       observer_body_name="Sun",
       reference_frame_name=global_frame_orientation,
       aberration_corrections="NONE",
       ephemeris_time= departure_epoch )
    
    final_state = spice_interface.get_body_cartesian_state_at_epoch(
       target_body_name= target_body,
       observer_body_name="Sun",
       reference_frame_name=global_frame_orientation,
       aberration_corrections="NONE",
       ephemeris_time= arrival_epoch )
    
    # Create Lambert targeter
    lambertTargeter = two_body_dynamics.LambertTargeterIzzo(
                initial_state[:3], final_state[:3],arrival_epoch - departure_epoch, central_body_gravitational_parameter );
    
    # Compute initial Cartesian state of Lambert arc
    lambert_arc_initial_state = initial_state
    lambert_arc_initial_state[3:] = lambertTargeter.get_departure_velocity()
    
    # Compute Keplerian state of Lambert arc
    lambert_arc_keplerian_elements = conversion.cartesian_to_keplerian( lambert_arc_initial_state, 
                                                                  central_body_gravitational_parameter)
    
    # Setup Keplerian ephemeris model that describes the Lambert arc
    kepler_ephemeris = environment_setup.create_body_ephemeris(
        environment_setup.ephemeris.keplerian( lambert_arc_keplerian_elements, departure_epoch, central_body_gravitational_parameter ), "" )
   
    return kepler_ephemeris

def get_lambert_arc_history( lambert_arc_state_model, simulation_result ):
    
    lambert_arc_states = dict()
    for state in simulation_result:
        lambert_arc_states[ state ] = lambert_arc_state_model.get_cartesian_state( state )
        
    return lambert_arc_states


def propagate_trajectory( initial_time, final_time, bodies, lambert_arc_state_model, 
                          file_output_identifier, use_perturbations, initial_state_correction=[0,0,0,0,0,0]):
    
    """
    This function will be repeatedly called throughout the assignment. Propagates the trajectory based 
    on several input parameters, and subsequently saves the results to data files.
    
    Parameters
    ----------
    initial_time : Epoch since J2000 at which the propagation starts
    
    final_time : Epoch since J2000 at which the propagation will be terminated
    
    lambert_arc_state_model : Lambert arc state model as returned by the get_lambert_problem_result() function
    
    file_output_identifier : Name that will be used to correctly save the output data files
    
    use_perturbations : Boolean to indicate whether a perturbed (True) or unperturbed (False) trajectory 
                        is propagated
                        
    initial_state_correction : (optional) Cartesian state which is added to the Lambert arc state when computing the numerical initial state
    
    Return
    ------
    Dynamics simulator object from which the state- and dependent variable history can be extracted
    
    """
    
    # Compute initial state along Lambert arc (and apply correction if needed)
    lambert_arc_initial_state = lambert_arc_state_model.get_cartesian_state( initial_time ) + initial_state_correction

    # Get propagator settings for perturbed/unperturbed forwards/backwards arcs
    if use_perturbations:
        propagator_settings = get_perturbed_propagator_settings( bodies, lambert_arc_initial_state, final_time )
    else:
        propagator_settings = get_unperturbed_propagator_settings( bodies, lambert_arc_initial_state, final_time )
           
    # If propagation is backwards in time, make initial time step negative
    if initial_time > final_time:
        signed_fixed_step_size = -fixed_step_size
    else:
        signed_fixed_step_size = fixed_step_size
        
    # Create numerical integrator settings
    integrator_settings = propagation_setup.integrator.runge_kutta_4( initial_time, signed_fixed_step_size )
    
    # Propagate forward/backward perturbed/unperturbed arc and save results to files
    dynamics_simulator = propagation_setup.SingleArcDynamicsSimulator(bodies, integrator_settings, propagator_settings, True)
    write_propagation_results_to_file( dynamics_simulator, lambert_arc_state_model, file_output_identifier, output_directory)

    return dynamics_simulator
  
def propagate_variational_equations(initial_time, final_time, bodies, lambert_arc_state_model):
    
    """
    Propagates the variational equations for a given range of epochs for a perturbed trajectory.
    
    Parameters
    ----------
    initial_time : Epoch since J2000 at which the propagation starts
    
    final_time : Epoch since J2000 at which the propagation will be terminated
    
    bodies : Body objects as returned by creates_simulation_bodies() function
    
    lambert_arc_state_model : Lambert arc state model as returned by the get_lambert_problem_result() function
    
    Return
    ------
    Variational equations solver object, from which the state-, state transition matrix-, and 
    sensitivity matrix history can be extracted.
    """
    
    # Compute initial state along Lambert arc
    lambert_arc_initial_state = lambert_arc_state_model.get_cartesian_state( initial_time )

    # Get propagator settings
    propagator_settings = get_perturbed_propagator_settings(bodies, lambert_arc_initial_state, final_time)
    
    # Get integrator settings       
    integrator_settings = propagation_setup.integrator.runge_kutta_4(
        initial_time, fixed_step_size )
    
    # Define parameters for variational equations
    sensitivity_parameters = get_sensitivity_parameter_set( propagator_settings, bodies, target_body)  
    
    # Propagate variational equations
    variational_equations_solver = estimation_setup.SingleArcVariationalEquationsSolver(
         bodies, integrator_settings, propagator_settings, sensitivity_parameters,integrate_on_creation=1 )
    
    return variational_equations_solver

def get_sensitivity_parameter_set(propagator_settings, bodies, target_body):

    parameter_settings = estimation_setup.parameter.initial_states(
        propagator_settings, bodies )
    if current_question == 4:
        parameter_settings.append( estimation_setup.parameter.constant_empirical_acceleration_terms ("Spacecraft", "Sun" ) )    
    
    return estimation_setup.create_parameters_to_estimate(parameter_settings, bodies, propagator_settings)

# Helper Functions (TO BE MODIFIED)

In [ ]:
def get_unperturbed_propagator_settings(bodies, initial_state, termination_time):
    
    """
    Creates the propagator settings for an unperturbed trajectory.

    Parameters
    ----------
    bodies : Body objects as returned by creates_simulation_bodies() function 
        
    initial_state : Cartesian initial state of the vehicle in the simulation
        
    termination_time : Epoch since J2000 at which the propagation will be terminated
        

    Return
    ------
    Propagation settings of the unperturbed trajectory.
    """

    bodies_to_propagate = ["Spacecraft"]
    central_bodies = ["Sun"]
    # Define accelerations acting on Vehicle
    accelerations_settings_vehicle = dict(
        Sun =
        [
            propagation_setup.acceleration.point_mass_gravity()
        ])

    # Create global accelerations settings dictionary.
    acceleration_settings = {"Spacecraft": accelerations_settings_vehicle}

    # Create acceleration models.
    acceleration_models = propagation_setup.create_acceleration_models(
        bodies, acceleration_settings, bodies_to_propagate, central_bodies)
    
    # Create propagation settings with termination time.
    propagator_settings = propagation_setup.propagator.translational(
    central_bodies,
    acceleration_models,
    bodies_to_propagate,
    initial_state,
    termination_time)
    
    
    return propagator_settings

# STUDENT CODE TASK - define function such that it provides propagator settings as per question 2-5
def get_perturbed_propagator_settings(bodies, initial_state, termination_time):
    
    """
    Creates the propagator settings for a perturbed trajectory.

    Parameters
    ----------
    bodies : Body objects as returned by creates_simulation_bodies() function 
        
    initial_state : Cartesian initial state of the vehicle in the simulation
        
    termination_time : Epoch since J2000 at which the propagation will be terminated
        

    Return
    ------
    Propagation settings of the perturbed trajectory.
    """

    bodies_to_propagate = ["Spacecraft"]
    central_bodies = ["Sun"]
    # Define accelerations acting on vehicle. 
    acceleration_settings_on_spacecraft = dict(
    Sun=
    [
        propagation_setup.acceleration.point_mass_gravity(),
        propagation_setup.acceleration.cannonball_radiation_pressure()
    ],
    Moon=
    [
        propagation_setup.acceleration.point_mass_gravity()
    ],
    Earth=
    [
        propagation_setup.acceleration.point_mass_gravity()
    ],
    Saturn=
    [
        propagation_setup.acceleration.point_mass_gravity()
    ],
    Jupiter=
    [
        propagation_setup.acceleration.point_mass_gravity()
    ],
    Venus=
    [
        propagation_setup.acceleration.point_mass_gravity()
    ]
    )
    
    
    # DO NOT MODIFY (line is added for compatibility with question 4)
    if current_question == 4:
        acceleration_settings_on_spacecraft[ "Sun" ].append( propagation_setup.acceleration.empirical( rsw_acceleration_magnitude ) )
    
    # Setup dependent variables
    dependent_variables_to_save = []
    if current_question == 2:
        dependent_variables_to_save = [
            propagation_setup.dependent_variable.total_acceleration( "Spacecraft" ),
        ]

    # Create global accelerations settings dictionary.
    acceleration_settings = {"Spacecraft": acceleration_settings_on_spacecraft}

    # Create acceleration models.
    acceleration_models = propagation_setup.create_acceleration_models(
        bodies, acceleration_settings, bodies_to_propagate, central_bodies)
    
    # Create propagation settings with termination time.
    propagator_settings = propagation_setup.propagator.translational(
    central_bodies,
    acceleration_models,
    bodies_to_propagate,
    initial_state,
    termination_time,
    output_variables = dependent_variables_to_save)
    
    return propagator_settings

# STUDENT CODE TASK - define function such that it creates the bodies needed for the simulation
def create_simulation_bodies():
    
    """
    Creates the body objects required for the simulation.
    Vehicle interfaces, such as the radiation pressure interface, will be defined here.

    Parameters
    ----------
    bodies_to_create - bodies to create in simulation

    Return
    ------
    Body objects required for the simulation.
    
    """
    # bodies_to_create = ["Mars", "Earth", "Sun"]
    bodies_to_create = ["Mars", "Earth", "Sun",  "Venus", "Moon", "Jupiter", "Saturn"]
    # Create body settings
    body_settings = environment_setup.get_default_body_settings(
        bodies_to_create, "SSB", "ECLIPJ2000")
    
    # Create environment of bodies
    bodies = environment_setup.create_system_of_bodies(body_settings)
    
    # Create vehicle object
    bodies.create_empty_body("Spacecraft")

    # Set mass of vehicle
    bodies.get_body("Spacecraft").set_constant_mass(1000.0)

    # Create radiation pressure interface
    reference_area = 20
    
    radiation_pressure_coefficient = 1.2
    occulting_bodies = []
    radiation_pressure_settings = environment_setup.radiation_pressure.cannonball(
    "Sun", reference_area, radiation_pressure_coefficient, occulting_bodies
    )
    environment_setup.add_radiation_pressure_interface(bodies, "Spacecraft", radiation_pressure_settings )
    
    return bodies

## Own written Functions

In [ ]:
def norm_manual(state, ind_r = [0, 1, 2]):
    nrows = np.shape(state)[0]
    raw_mat = state[:,ind_r]
    normalized_mat = np.zeros((nrows, 3))  
    len_mat = np.zeros((nrows, 1))  
    for ii in range(nrows):
        len_mat_ii = np.sqrt(raw_mat[ii,0]**2 + raw_mat[ii,1]**2 + raw_mat[ii,2]**2)
        raw_mat_ii = raw_mat[ii,:]
        normalized_mat_ii = raw_mat_ii / len_mat_ii
        # Store
        normalized_mat[ii,:] = normalized_mat_ii
        len_mat[ii] = len_mat_ii
    return normalized_mat, len_mat

def delta_pos_manual(state_i, state_ii, ind_r = [0, 1, 2]):
    """Function that calculates the offsets of each vector component and their norms
    Args:
        state_i (array): states of the first body
        state_ii (array): states of the second body
        pos_ind (list, optional): Indices to the position vector columns in the state matrices. Defaults to [0, 1, 2].
    Returns:
        [type]: [description]
    """
    import numpy as np
    r_i = state_i[:,ind_r]
    r_ii = state_ii[:,ind_r]
    nrows = np.shape(r_i)[0]
    norm_delta_r = np.zeros((nrows, 1))
    delta_r = r_i - r_ii
    for ii in range(nrows):
        delta_r_ii = delta_r[ii, :]
        norm_delta_r_ii = np.sqrt(delta_r_ii[0]**2 + delta_r_ii[1]**2 + delta_r_ii[2]**2)
        norm_delta_r[ii] = norm_delta_r_ii
    return (delta_r, norm_delta_r)

def multicol_multiplot(dat1, dat2, dat3, plot_title = '', x_label = 'Time since start [Days]', y_label = [r'$\Delta$r[m]', r'$\Delta$v[m/s]', r'$\Delta$a[m/s$^2$]'], labels = [r'$case_i$', r'$case_{ii}$', r'$case_{iii}$'], log = [True, False, False]):
    # Prepare vectors for plotting - 3 rows with multiple lines
    dat1[:,0] = (dat1[:,0] - dat1[0,0])/constants.JULIAN_DAY
    dat2[:,0] = (dat2[:,0] - dat2[0,0])/constants.JULIAN_DAY
    dat3[:,0] = (dat3[:,0] - dat3[0,0])/constants.JULIAN_DAY
    
    # Make figure
    fig, axs = plt.subplots(nrows = 3, ncols = 1, figsize = (14, 15))
    colors = ['r', 'g', 'b']
    
    # Plot data
    for ii in range(3):
        axs[ii].plot(dat1[:,0], dat1[:,ii+1], label = labels[0])
        axs[ii].plot(dat2[:,0], dat2[:,ii+1], label = labels[1])
        axs[ii].plot(dat3[:,0], dat3[:,ii+1], label = labels[2])
    
    # Glossary
    for ii in range(3):
        if log[ii]:
            axs[ii].set_yscale('log')
        axs[ii].grid('on')
        axs[ii].set_ylabel(y_label[ii], fontweight='bold', size=10)
        axs[ii].set_xlim(min(dat1[:,0]-5), max(dat1[:,0]))
        axs[ii].set_xlabel(x_label, fontweight='bold', size=10)
        axs[ii].legend(shadow=True, fancybox=True)
    
    return fig, axs

def doublecol_multiplot(dat1, dat2, initial_time, plot_title = '', x_label = 'Time since start [Days]', y_label = [r'$\Delta$r[m]', r'$\Delta$v[m/s]', r'$\Delta$a[m/s$^2$]'], labels = [r'$case_{ii}$ backward', r'$case_{ii} $forward'], log = [False, False, False]):
    # Prepare vectors for plotting - 3 rows with 2 data lines
    dat1[:,0] = (dat1[:,0] - initial_time)/constants.JULIAN_DAY
    dat2[:,0] = (dat2[:,0] - initial_time)/constants.JULIAN_DAY
    
    ncols = 3
    
    # Get x_limits
    x_limits = [np.min(dat1[:,0]-5), np.max(dat2[:,0])]
    
    # Make figure
    fig, axs = plt.subplots(nrows = ncols, ncols = 1, figsize = (14, 15))
    colors = ['r', 'g', 'b']
    
    # Plot data
    for ii in range(ncols):
        axs[ii].plot(dat1[:,0], dat1[:,ii+1], label = labels[0])
        axs[ii].plot(dat2[:,0], dat2[:,ii+1], label = labels[1])
    
    # Glossary
    for ii in range(ncols):
        if log[ii]:
            axs[ii].set_yscale('log')
        axs[ii].grid('on')
        axs[ii].set_ylabel(y_label[ii], fontweight='bold', size=10)
        axs[ii].set_xlim(x_limits)
        axs[ii].set_xlabel(x_label, fontweight='bold', size=10)
        axs[ii].legend(shadow=True, fancybox=True)
    
    return fig, axs

def multicol_multiplot(dat1, dat2, dat3, plot_title = '', x_label = 'Time since start [Days]', y_label = [r'$\Delta$r[m]', r'$\Delta$v[m/s]', r'$\Delta$a[m/s$^2$]'], labels = [r'$case_i$', r'$case_{ii}$', r'$case_{iii}$'], log = [True, False, False]):
    # Prepare vectors for plotting - 3 rows with multiple lines
    dat1[:,0] = (dat1[:,0] - dat1[0,0])/constants.JULIAN_DAY
    dat2[:,0] = (dat2[:,0] - dat2[0,0])/constants.JULIAN_DAY
    dat3[:,0] = (dat3[:,0] - dat3[0,0])/constants.JULIAN_DAY
    
    # Make figure
    fig, axs = plt.subplots(nrows = 3, ncols = 1, figsize = (14, 15))
    colors = ['r', 'g', 'b']
    
    # Plot data
    for ii in range(3):
        axs[ii].plot(dat1[:,0], dat1[:,ii+1], label = labels[0])
        axs[ii].plot(dat2[:,0], dat2[:,ii+1], label = labels[1])
        axs[ii].plot(dat3[:,0], dat3[:,ii+1], label = labels[2])
    
    # Glossary
    for ii in range(3):
        if log[ii]:
            axs[ii].set_yscale('log')
        axs[ii].grid('on')
        axs[ii].set_ylabel(y_label[ii], fontweight='bold', size=10)
        axs[ii].set_xlim(min(dat1[:,0]-5), max(dat1[:,0]))
        axs[ii].set_xlabel(x_label, fontweight='bold', size=10)
        axs[ii].legend(shadow=True, fancybox=True)
    
    return fig, axs

def delta_pos_plot(dat1, initial_time, dat2 = None, plot_title = '', x_label = 'Time since start [Days]', y_label = [r'$\Delta$r[m]'], labels = [r'Not corrected', r'Corrected'], log = [True, False, False]):
    # Single row plot- single or two lines
    dat1[:,0] = (dat1[:,0] - initial_time)/constants.JULIAN_DAY
    if type(dat2) != type(None):
        dat2[:,0] = (dat2[:,0] - initial_time)/constants.JULIAN_DAY
    ncols = 1
    # Get x_limits
    x_limits = [np.min(dat1[:,0]-5), np.max(dat1[:,0])]
    
    # Make figure
    fig, ax = plt.subplots(nrows = ncols, ncols = 1, figsize = (5,5))
    colors = ['r', 'g', 'b']
    
    # Plot data
    for ii in range(ncols):
        ax.plot(dat1[:,0], dat1[:,ii+1], label = labels[0])
        if type(dat2) != type(None):
            ax.plot(dat2[:,0], dat2[:,ii+1], label = labels[1 ])
        # axs[ii].plot(dat2[:,0], dat2[:,ii+1], label = labels[1])
    
    # Glossary
    for ii in range(ncols):
        if log[ii]:
            ax.set_yscale('log')
        ax.grid('on')
        ax.set_ylabel(y_label[ii], fontweight='bold', size=10)
        ax.set_xlim(x_limits)
        ax.set_xlabel(x_label, fontweight='bold', size=10)
        ax.legend(shadow=True, fancybox=True)
    
    return fig, ax

def delta_pos_plot_unlimited(dat_list, initial_time, plot_title = '', x_label = 'Time since start [Days]', y_label = [r'$\Delta$r[m]'], labels = [r'Not corrected', r'Corrected'], log = [True, False, False]):
    # Single row plot- list of lines
    t_vec_list = []
    for dat in dat_list: # Convert to days since start of propagation
        t_vec_list.append((dat[:,0] - initial_time)/constants.JULIAN_DAY)
    ncols = 1
    # Get x_limits
    x_limits = [np.min(t_vec_list[0]-5), np.max(t_vec_list[0])]
    
    # Make figure
    fig, ax = plt.subplots(nrows = ncols, ncols = 1, figsize = (5,5))
    colors = ['r', 'g', 'b']
    
    # Plot data
    # for ii in range(ncols):
    for jj in range(len(dat_list)):
        dat = dat_list[jj]
        ax.plot(t_vec_list[jj], dat[:,1], label = labels[jj])
    
    # Glossary
    for ii in range(ncols):
        if log[ii]:
            ax.set_yscale('log')
        ax.grid('on')
        ax.set_ylabel(y_label[ii], fontweight='bold', size=10)
        ax.set_xlim(x_limits)
        ax.set_xlabel(x_label, fontweight='bold', size=10)
        ax.legend(shadow=True, fancybox=True)
    
    return fig, ax

def delta_vel_plot_unlimited(dat_list, initial_time, plot_title = '', x_label = 'Time since start [Days]', y_label = [r'$\Delta$v[m/s]'], labels = [r'Not corrected', r'Corrected'], log = [True, False, False]):
    # Single row plot- list of lines
    t_vec_list = []
    for dat in dat_list: # Convert to days since start of propagation
        t_vec_list.append((dat[:,0] - initial_time)/constants.JULIAN_DAY)
    # print(t_vec_list)
    ncols = 1
    # Get x_limits
    x_limits = [np.min(t_vec_list[0]-5), np.max(t_vec_list[0])]
    
    # Make figure
    fig, ax = plt.subplots(nrows = ncols, ncols = 1, figsize = (5,5))
    colors = ['r', 'g', 'b']
    
    # Plot data
    # for ii in range(ncols):
    for jj in range(len(dat_list)):
        dat = dat_list[jj]
        ax.plot(t_vec_list[jj], dat[:,2], label = labels[jj])
    
    # Glossary
    for ii in range(ncols):
        if log[ii]:
            ax.set_yscale('log')
        ax.grid('on')
        ax.set_ylabel(y_label[ii], fontweight='bold', size=10)
        ax.set_xlim(x_limits)
        ax.set_xlabel(x_label, fontweight='bold', size=10)
        ax.legend(shadow=True, fancybox=True)
    
    return fig, ax

def combine_arcs(arc_list):
    # Function used to combine multiple arc matrices stored in a list into a single matrix
    arc_combined = arc_list[0] # make placeholder
    for ii in range(1, len(arc_list)):
        arc_combined = np.vstack((arc_combined, arc_list[ii]))
    return arc_combined
def find_peak(y_data, x_data):
    """Function to return the maximum value and the time when it occurs

    Args:
        y_data ([type]): [description]
        x_data ([type]): [description]

    Returns:
        [type]: (y_data[ind_max], x_data[ind_max] - x_data[0])
    """    # 
    ind_max = np.where(y_data == np.max(y_data))[0][0]
    return (y_data[ind_max], x_data[ind_max] - x_data[0])
    

def orbitx2_3d(state_i, state_ii, ind_r = [0, 1, 2], label1_f = 'Object i', label2_f = 'Object ii', title_f = '', R_P = 696.34e6, P_f = 'Sun'):
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt	
    # Function used to plot a 3d orbit of a spacecraft around the body p
    fig = plt.figure(figsize = (15,15))
    ax = fig.add_subplot(111, projection='3d')
    # Unpack states
    X_f = state_i[:,ind_r[0]]
    Y_f = state_i[:,ind_r[1]]
    Z_f = state_i[:,ind_r[2]]
    X_f2 = state_ii[:,ind_r[0]]
    Y_f2 = state_ii[:,ind_r[1]]
    Z_f2 = state_ii[:,ind_r[2]]
    # Draw planet sphere
    if len(P_f)>0: # if name of planet is provided, plot it
        # draw sphere
        u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
        x = np.cos(u)*np.sin(v) * R_P
        y = np.sin(u)*np.sin(v) * R_P 
        z = np.cos(v) * R_P 
        ax.plot_wireframe(x, y, z, color="r", label = P_f + ' sphere') # plot the planet
    #set plot limits
    r_lim = np.linalg.norm((X_f[1], Y_f[1], Z_f[1]))
    limits= [-r_lim*2,r_lim*2]
    ax.set_xlim(limits)
    ax.set_ylim(limits)
    ax.set_zlim(limits)
    # Plot first trajectory
    ax.plot(X_f, Y_f, Z_f, label = label1_f)
    # Plot second trajectory
    ax.plot(X_f2, Y_f2, Z_f2, label = label2_f)
    plt.title(title_f, fontsize = 16, fontweight = 'bold')
    ax.set_xlabel('X [m]', fontsize = 14, fontweight = 'bold')
    ax.set_ylabel('Y [m]', fontsize = 14, fontweight = 'bold')
    ax.set_zlabel('Z [m]', fontsize = 14, fontweight = 'bold')
    ax.legend()
    plt.grid()
    plt.legend()
    return fig, ax

def savefig(fig, name, save_folder_figures = r"C:\Users\Beautiful\Documents\Github Repositories\NumericalAstrodynamics2020_Kipras\Assignment2\Figures"):
	fig.savefig(save_folder_figures + "//" + name + '.png', bbox_inches='tight')
 
def open_dat(filename = 0, folderPath = r"C:\Users\Beautiful\Documents\Github Repositories\NumericalAstrodynamics2020_Kipras\Assignment2\SimulationOutput", file_index = -1, first_row = 0, debug = 0, print_cond = 1):
    import os
    try: # Choosing the correct file
        filename_n_path = folderPath + "\\" + filename
        data_raw = open(filename_n_path).readlines()
        if print_cond:
            print("Loaded file ", filename)
        # , 'modified on', time.ctime(os.path.getmtime(filename_n_path))[:3], ',', time.ctime(os.path.getmtime(filename_n_path))[8:-4])
        data_content = np.array([i.strip().split(',') for i in data_raw[first_row:][1:]]).astype(float)
        return data_content
    except:
        if print_cond:
            print('Unable to load %s' % filename_n_path)
def open_arc(arc_name_0, num_arcs, print_cond):
    # function to open num_arc dat files and load them into a matrix of matrices
    output_list = [] # Used to store output matrices
    if num_arcs>1:
        ind_split = arc_name_0.index('0')
        name_first = arc_name_0[:ind_split]
        name_final = arc_name_0[ind_split+1:]       
        for ii in range(num_arcs):
            arc_name = name_first + str(int(ii)) + name_final
            arc_data = open_dat(arc_name, print_cond = print_cond)
            output_list.append(arc_data)
        if print_cond:
            print('Loaded %s' % name_first+name_final)
    else:
        arc_data = open_dat(arc_name_0, print_cond = print_cond)
        output_list.append(arc_data)
    # print('Length of output list', len(output_list))
    return output_list

def delta_data_matrix_from_arcs(arc_name, num_arcs = 2, print_condition = 1):
    """Function to completely perform the delta_r calculation and output a t, dr, dv matrix`

    Args:
        arc_name (string): first part of arc name ("Q3_arc_0_corrected as an example)
        num_arcs (int, optional): Number of arcs used in simulation. Defaults to 2.
    """
        
    # Load data files, two-arc simulation
    # arc_name = Q3_arc_0_corrected
    arc_lam = arc_name + '_lambert_states.dat' 
    arc_num = arc_name + '_numerical_states.dat' 
    s_lam_fct_raw_single = open_arc(arc_lam, num_arcs, print_condition)
    s_num_fct_raw_single = open_arc(arc_num, num_arcs, print_condition)
    if print_condition:
        print('Loaded ', arc_lam, ' and ', arc_num)
        print('Size %d x %d' % (np.shape(s_lam_fct_raw_single[0])[0], np.shape(s_lam_fct_raw_single[0])[1]))
    # Combine into single matrix
    s_lam_fct_single = combine_arcs(s_lam_fct_raw_single)
    s_num_fct_single = combine_arcs(s_num_fct_raw_single)
    if print_condition:
        print('Combined shape %d x %d' % (np.shape(s_lam_fct_single)[0], np.shape(s_lam_fct_single)[1]))
    # Get time vector
    time_vec_fct_single = s_num_fct_single[:,[0]]
    ind_r = [1,2,3]
    ind_v = [4,5,6]
    delta_r_fct_single = delta_pos_manual(s_lam_fct_single, s_num_fct_single, ind_r)[1]
    delta_v_fct_single = delta_pos_manual(s_lam_fct_single, s_num_fct_single, ind_v)[1]
    # Prepare data matrix
    fct_single_mat = np.hstack((time_vec_fct_single, delta_r_fct_single, delta_v_fct_single))
    return fct_single_mat

def calculate_epsilon(lambert_states, numerical_states, STM_states, delta_x0):
    """Calculates the epsilon values for a given lambert state model, 
    perturbed numerical model and state transition matrix history 
    + change in initial conditions

    Args:
        lambert_states (dict): Lambert states dictionary
        numerical_states (dict): perturbed numerical states dict
        STM_states (dict): State transition matrix dict
        delta_x0 (6x1 array): analyzed change in initial conditions

    Returns:
        arrays: 0- epsilon matrix. 1- delta x. 2- linearized delta x
    """    
    time_vec = list(numerical_states.keys())
    
    # Get numerical delta_x
    delta_x = [numerical_states[time] - lambert_states[time]  for time in time_vec]
    
    # Get linearized delta_x
    delta_x_linearized = [np.transpose(np.matmul(STM_states[time], delta_x0))  for time in time_vec]
    
    # get Epsilon_x
    # epsilon_x = [delta_x_linearized[ii] - delta_x[ii] for ii in range(len(delta_x_linearized))]
    # epsilon_x = delta_x_linearized - delta_x
    epsilon_x = np.zeros((len(delta_x_linearized), 6))
    for ii in range(len(delta_x_linearized)):
        # print(delta_x_linearized[ii])
        # print(delta_x[ii])
        epsilon_x[ii,:] = delta_x_linearized[ii] - delta_x[ii]
    return epsilon_x, delta_x, delta_x_linearized

def check_condition(epsilon_matrix, r_max = 100e3, v_max = 1, r_ind = [0,1,2], v_ind = [3,4,5]):    
    """Function to check if the deviation condition has been triggerred. Returns True if one of the epsilons is too large

    Args:
        epsilon_matrix (array): epsilon matrix N x 6
        r_max (int, optional): maximum allowed delta r. Defaults to 100e3.
        v_max (int, optional): maximum allowed delta v. Defaults to 1.
        r_ind (list, optional): inidices corresponding to eps_r. Defaults to [0,1,2].
        v_ind (list, optional): inidices corresponding to eps_v. Defaults to [3,4,5].

    Returns:
        list of bools, array, array: [if conditions were met, computed ||epsilon_r|| and ||epsilon_v||]
    """    
    cond_r = 0
    cond_v = 0
    
    epsilon_r = norm_manual(epsilon_matrix, r_ind)[1]
    epsilon_v = norm_manual(epsilon_matrix, v_ind)[1]
    
    if np.max(epsilon_r) > r_max:
        cond_r = 1
    if np.max(epsilon_v) > v_max:
        cond_v = 1

    return [cond_r, cond_v], np.max(epsilon_r), np.max(epsilon_v)

def check_convergence(fractions = [2, 0], allowed_offset = 0.01):
    
    fraction_1 = fractions[0]
    fraction_2 = fractions[1]
    
    if fraction_1<1: # if fraction 2 is within 1% of set limit
        if fraction_2<1.5: # check if fraction_2 is within allowed offset of the limit
            if np.abs(fraction_2 - 1 ) <= allowed_offset:
                return True
    elif fraction_2 < 1: # if fraction 1 is within 1% of set limit
        if fraction_1 < 1.5: # check if fraction_1 is within allowed offset of the limit
            if np.abs(fraction_1 - 1 ) <= allowed_offset:
                return True
    else: # if both fractions are within 1% of set limit
        if np.abs(fraction_1 - 1 ) <= allowed_offset and np.abs(fraction_2 - 1 ) <= allowed_offset:
            return True
    return False # if no condition is trigerred 

# Question 1 
### 10 points; Maximum text length: 10 lines

Use the Lambert arc's initial state $\bar{\mathbf{x}}(t_{0})$ as initial state for the numerical propagation (spacecraft w.r.t. Sun), with the given initial time $t_{0}$. Run the code to propagate the state of the spacecraft using only the Sun's point-mass attraction, with the Sun as propagation origin. 
<ul>
    <li> Plot the total trajectory in three dimensions. </li>
    <li> Plot the difference between the Lambert targeter result $\bar{\mathbf{x}}(t)$ and the numerical propagation ${\mathbf{x}}(t)$. Specifically, plot the difference in each of the three Cartesian position components of the spacecraft w.r.t. the Sun.</li>
</ul>
    

<style type="text/css">
    ol { list-style-type: upper-alpha; }
</style>

**Answer the following questions:**

**a)** Discuss whether the mathematical model solved numerically in this question is an *exact* representation of the Lambert arc model. In your discussion, add a comprehensive list of the assumptions of the Lambert arc model, and briefly explain for each one why it is (not) true in your propagation. If needed, back up your argumentation by adding, and plotting, any additional dependent variables to the simulation output you see fit.

**b)** If you concluded under (a) that the formulations are physically identical, what is the source of any differences you observe between the numerical result and the Lambert result? If you concluded that they are not identical, is any difference you observe relevant for your results?

**Add to save file 1**: <br>
Row 1: initial propagation time and Cartesian state. <br>
Row 2: final propagation time and Cartesian state.

**Coding instructions and hints**

The code block below propagates the dynamics for *unperturbed* dynamics, with the initial state extracted directly from the Lambert arc. The resulting numerical state history ($\mathbf{x}(t)$) is stored in the `state_history` variable. The state history, as computed directly from the Lambert arc ($\bar{\mathbf{x}}(t)$) is stored directly in the `lambert_history` variable.

To make this function generate results, you have a **Code task** for the following helper functions:

* `create_simulation_bodies`: this function defines the body settings and creates the body objects
* `get_unperturbed_propagator_settings`: this function defines the propagator settings for the unperturbed case

In order to ensure compatibility with the rest of the code, call your vehicle:** `"Spacecraft"`. 

In [ ]:
###########################################################################
# RUN CODE FOR QUESTION 1 #################################################
###########################################################################

current_question = 1
# Create body objects
bodies = create_simulation_bodies()

# Create Lambert arc state model
lambert_arc_state_model = get_lambert_problem_result(bodies, target_body, departure_epoch, arrival_epoch)

# Create propagation settings and propagate dynamics
dynamics_simulator = propagate_trajectory( departure_epoch, arrival_epoch, bodies, lambert_arc_state_model, 
                    "Q1", use_perturbations = False)

# Extract state history from dynamics simulator
state_history = dynamics_simulator.state_history

# Evaluate the Lambert arc model at each of the epochs in the state_history variable
lambert_history = get_lambert_arc_history( lambert_arc_state_model, state_history )

# Get states
r_lam = np.array(list(lambert_history.values()))
t_lam = np.array(list(lambert_history.keys()))
t_lam = np.reshape(t_lam, (np.shape(t_lam)[0], 1))
s_lam = np.hstack((t_lam, r_lam))

r_num = np.array(list(state_history.values()))
t_num = np.array(list(state_history.keys()))
t_num = np.reshape(t_num, (np.shape(t_num)[0], 1))
s_num = np.hstack((t_num, r_num))
## Make plots
fig_q1_dr, ax_q1_dr = delta_pos_plot(r_lam, r_num, t_lam, 'Difference in position between the Lambert arc and unperturbed orbit')
savefig(fig_q1_dr, 'q1_delta_r')

fig_q1_3d, ax_q1_3d = orbitx2_3d(r_lam, r_num, label1_f = 'Lambert arc', label2_f = 'Numerically propagated orbit')
savefig(fig_q1_3d, 'q1_3d')

# Question 2 
### 20 points; Maximum text length: 15 lines

Now add the following perturbations to the numerical propagation (by modifying the `get_perturbed_propagation_settings` and the `create_simulation_bodies` functions):

<ul>
    <li> Point-mass gravity by Venus, Earth, Moon, Mars, Jupiter, Saturn and Sun </li>
    <li> Cannonball radiation pressure on spacecraft. Use a reference area of 20 m$^2$, a radiation pressure coefficient of 1.2, and a vehicle mass of 1000 kg. </li>
</ul>

To do so, also update the definition of the environment (modify the `create_simulation_bodies` function) so that these accelerations can be evaluated

Run the propagation with$^2$:<br>
i) The initial and final propagation time equal to the initial and final times of the Lambert arc. <br>
ii) The initial and final propagation time shifted forward and backward in time, respectively, by $\Delta t=$1 hour. <br>
iii) The initial and final propagation time shifted forward and backward in time, respectively, by $\Delta t=$2 days.

    
Note that if you shift the initial time from $t_{0}$ to $t_{0}+\Delta t$ (by modifying these inputs to the `propagate_trajectory` function), the initial state used for the propagation will be adjusted (automatically) accordingly to $\mathbf{\bar{x}}(t_{0}+\Delta t)$. 

For each propagation, plot the quantities $\Delta r=||\mathbf{r}(t)-\bar{\mathbf{r}}(t)||$, $\Delta v=||\mathbf{v}(t)-\bar{\mathbf{v}}(t)||$ and $\Delta a=||\mathbf{a}(t)-\bar{\mathbf{a}}(t)||$ as a function of time. **Note** The quantity $\bar{\mathbf{a}}(t)$ is not provided automatically, compute its value at each step manually from $\frac{-\mu}{r^{3}}\mathbf{r}$.

**Answer the following questions:**

**a)** For each of the cases (i)-(iii), find the maxima of each of the quantities $\Delta r$, $\Delta v$ and $\Delta a$, and the times at which they occur. Provide a table with the following for each of the cases (i)-(iii): the maximum values of these three quantities, and the times at which they reach their maxima. Write these times *as a fraction of the total propagation time.* Note that a visual determination of the required quantities from the plots you make is sufficient for the purposes of this question.
    
**b)** Redo the propagation for case (ii), but now propagating forward and backward from the middle point (in time) of the propagation. Plot the quantities quantities $\Delta r$, $\Delta v$ and $\Delta a$, and extend the table you made in question (a) with the results of this question. Consider the forward and backward propagation separately. *Note: the* `propagate_trajectory` *function will automatically propagate backwards in time if the initial time is larger than the final time.*
    
**c)** Analytically derive formulations for $\frac{d}{dt}(\Delta r)$ and $\frac{d}{dt}(\Delta v)$. These quantities are a measure for how quickly the numerical solution diverges from the Lambert arc solution. In your derivations, use only the following quantities:

<ul>
<li> Positions and velocities along the propagated orbit $\mathbf{r}$, $\mathbf{v}$
<li> Positions and velocities along the Lambert arc $\bar{\mathbf{r}}$, $\bar{\mathbf{v}}$
<li> Total accelerations in the Lambert model  $\bar{\mathbf{a}}$
<li> The acceleration components $\mathbf{a}_{\text{Sun}}$ and $\mathbf{a}_{\text{pert}}$ of the propagated orbit.
</ul>
In the above, we have used (for the numerical model) the shorthand $\mathbf{a}_{\text{Sun}}(t)$ for the Sun's gravitational acceleration, and have lumped all additional accelerations  into $\mathbf{a}_{\text{pert}}$, so that the total acceleration acting on the spacecraft is given by $\left(\mathbf{a}_{p}\right)_{S}(t)=\mathbf{a}_{\text{Sun}}(t)+\mathbf{a}_{\text{pert}}(t)$. 

**Hint**: Use the relation $\dfrac{d||\mathbf{b}||}{d\mathbf{b}}=\dfrac{\mathbf{b}^{T}}{||\mathbf{b}||}$, for an arbitrary vector $\mathbf{b}$.

**d)**  Use the equations derived in (c), and the table constructed in (a) and (b) to explain why $\Delta r$ behaves very differently in the cases that are analyzed. Specifically, explain:

<ul>
<li> Why increasing the buffer time $\Delta t$ seems to generally reduce the magnitude of $\Delta r$ (question a)
<li> Why starting the propagation at middle of the arc, and propagating forwards and backwards, results in a smaller maximum value $\Delta r$ then only propagating forward, even for equal $\Delta t$ (question (b) vs. case ii in question (a)).
</ul>
  
**Add to save file 1** <br>
Row 3: initial propagation time and Cartesian state (case i). <br>
Row 4: final propagation time and Cartesian state (case i). <br>
Row 5: initial propagation time and Cartesian state (case ii, question a). <br>
Row 6: final propagation time and Cartesian state (case ii, question a). <br>
Row 7: initial propagation time and Cartesian state (case iii). <br>
Row 8: final propagation time and Cartesian state (case iii). <br>


**Coding instructions and hints**

The code block below propagates the dynamics for *perturbed* dynamics, with the initial state extracted directly from the Lambert arc. The resulting numerical state history ($\mathbf{x}(t)$) is stored in the `state_history` variable. The state history, as computed directly from the Lambert arc ($\bar{\mathbf{x}}(t)$) is stored directly in the `lambert_history` variable.

To make this function generate results, you have a **Code task** for the following helper functions:

* `create_simulation_bodies`: this function defines the body settings and creates the body objects
* `get_perturbed_propagator_settings`: this function defines the propagator settings for the unperturbed case

To propagate the dynamics, do not create a `SingleArcDynamicsSimulator` manually, but make use of the `propagate_trajectory` function.

In order to ensure compatibility with the rest of the code, call your vehicle:** `"Spacecraft"`. 

In [ ]:
###########################################################################
# RUN CODE FOR QUESTION 2 #################################################
###########################################################################

current_question = 2

# Create body objects
bodies = create_simulation_bodies( )

# Create Lambert arc state model
lambert_arc_state_model = get_lambert_problem_result(bodies, target_body, departure_epoch, arrival_epoch)

"""
case_i: The initial and final propagation time equal to the initial and final times of the Lambert arc.
case_ii: The initial and final propagation time shifted forward and backward in time, 
respectively, by ∆t=1 hour.
case_iii: The initial and final propagation time shifted forward and backward in time,
respectively, by ∆t=2 days.

"""
cases = ['case_i', 'case_ii', 'case_iii']

# Define buffer times for each case
buffer_times = [0.0, 3600.0, 2.0 * constants.JULIAN_DAY]

# Run propagation for each of cases i-iii
for case in cases:
    
    # Compute departure and arrival time
    current_buffer_time = buffer_times[ cases.index(case) ]
    
    departure_epoch_with_buffer = departure_epoch + current_buffer_time
    arrival_epoch_with_buffer = arrival_epoch - current_buffer_time
    
    # Perform propagation
    # STUDENT CODE TASK (call propagation function)
    dynamics_simulator = propagate_trajectory( departure_epoch_with_buffer, arrival_epoch_with_buffer, bodies, lambert_arc_state_model, 
    "Q2_" + case, use_perturbations = True)
    state_history = dynamics_simulator.state_history
    lambert_history = get_lambert_arc_history( lambert_arc_state_model, state_history )
    
    # For case ii, run propagation forward and backwatd from mid-point
    if case == 'case_ii':
        
        # STUDENT CODE TASK (fill in ...)
        mid_point_epoch = (departure_epoch_with_buffer + arrival_epoch_with_buffer) / 2
        
        # Perform propagation forwards
        # STUDENT CODE TASK (call propagation function)
        dynamics_simulator = propagate_trajectory( mid_point_epoch, arrival_epoch_with_buffer, bodies, lambert_arc_state_model, 
    "Q2_midpoint_forward" + case, use_perturbations = True)
        state_history_forward = dynamics_simulator.state_history
        lambert_history_forward = get_lambert_arc_history( lambert_arc_state_model, state_history_forward )
        
        # Perform propagation backwards
        # STUDENT CODE TASK (call propagation function)
        dynamics_simulator = propagate_trajectory( mid_point_epoch, departure_epoch_with_buffer, bodies, lambert_arc_state_model, 
    "Q2_midpoint_backward" + case, use_perturbations = True)
        state_history_backward = dynamics_simulator.state_history
        lambert_history_backward = get_lambert_arc_history( lambert_arc_state_model, state_history_backward )

In [ ]:
## Plots - Question 2
# Open required data Case i
dep_var_q2_i = open_dat(filename = "Q2_case_i_dependent_variables.dat")
s_lam_q2_i = open_dat(filename = "Q2_case_i_lambert_states.dat")
s_num_q2_i = open_dat(filename = "Q2_case_i_numerical_states.dat")
# set up lists of indices
ind_r = [1,2,3]
ind_v = [4,5,6]
# get time
time_vec_i = s_num_q2_i[:,[0]]
# get delta_r
delta_r_i = delta_pos_manual(s_lam_q2_i, s_num_q2_i, ind_r)[1]
# get delta_v
delta_v_i = delta_pos_manual(s_lam_q2_i, s_num_q2_i, ind_v)[1]
# get delta_a
gravitational_parameter_sun = bodies.get_body( "Sun" ).gravitational_parameter
r_lam_norm_i, r_lam_len_i = norm_manual(s_lam_q2_i, ind_r)
a_lam_i = - gravitational_parameter_sun / r_lam_len_i**3  * r_lam_norm_i
a_num_i = dep_var_q2_i[:,ind_r]
delta_a_i = delta_pos_manual(a_lam_i, a_num_i, [0,1,2])[1]

# Open required data Case ii
dep_var_q2_ii = open_dat(filename = "Q2_case_ii_dependent_variables.dat")
s_lam_q2_ii = open_dat(filename = "Q2_case_ii_lambert_states.dat")
s_num_q2_ii = open_dat(filename = "Q2_case_ii_numerical_states.dat")
# set up lists of indices
ind_r = [1,2,3]
ind_v = [4,5,6]
# get time
time_vec_ii = s_num_q2_ii[:,[0]]
# get delta_r
delta_r_ii = delta_pos_manual(s_lam_q2_ii, s_num_q2_ii, ind_r)[1]
# get delta_v
delta_v_ii = delta_pos_manual(s_lam_q2_ii, s_num_q2_ii, ind_v)[1]
# get delta_a
gravitational_parameter_sun = bodies.get_body( "Sun" ).gravitational_parameter
r_lam_norm_ii, r_lam_len_ii = norm_manual(s_lam_q2_ii, ind_r)
a_lam_ii = - gravitational_parameter_sun / r_lam_len_ii**3  * r_lam_norm_ii
a_num_ii = dep_var_q2_ii[:,ind_r]
delta_a_ii = delta_pos_manual(a_lam_ii, a_num_ii, [0,1,2])[1]

# Open required data Case iii
dep_var_q2_iii = open_dat(filename = "Q2_case_iii_dependent_variables.dat")
s_lam_q2_iii = open_dat(filename = "Q2_case_iii_lambert_states.dat")
s_num_q2_iii = open_dat(filename = "Q2_case_iii_numerical_states.dat")
# set up lists of indices
ind_r = [1,2,3]
ind_v = [4,5,6]
# get time
time_vec_iii = s_num_q2_iii[:,[0]]
# get delta_r
delta_r_iii = delta_pos_manual(s_lam_q2_iii, s_num_q2_iii, ind_r)[1]
# get delta_v
delta_v_iii = delta_pos_manual(s_lam_q2_iii, s_num_q2_iii, ind_v)[1]
# get delta_a
gravitational_parameter_sun = bodies.get_body( "Sun" ).gravitational_parameter
r_lam_norm_iii, r_lam_len_iii = norm_manual(s_lam_q2_iii, ind_r)
a_lam_iii = - gravitational_parameter_sun / r_lam_len_iii**3  * r_lam_norm_iii
a_num_iii = dep_var_q2_iii[:,ind_r]
delta_a_iii = delta_pos_manual(a_lam_iii, a_num_iii, [0,1,2])[1]

case_iii_mat = np.hstack((time_vec_iii, delta_r_iii, delta_v_iii, delta_a_iii))
case_ii_mat = np.hstack((time_vec_ii, delta_r_ii, delta_v_ii, delta_a_ii))
case_i_mat = np.hstack((time_vec_i, delta_r_i, delta_v_i, delta_a_i))

# Open required data Case ii - backwards
dep_var_q2_ii_backwards = open_dat(filename = "Q2_midpoint_backwardcase_ii_dependent_variables.dat")
s_lam_q2_ii_backwards = open_dat(filename = "Q2_midpoint_backwardcase_ii_lambert_states.dat")
s_num_q2_ii_backwards = open_dat(filename = "Q2_midpoint_backwardcase_ii_numerical_states.dat")
# # set up lists of indices
ind_r = [1,2,3]
ind_v = [4,5,6]
# get time
time_vec_ii_backwards = s_num_q2_ii_backwards[:,[0]]
# get delta_r
delta_r_ii_backwards = delta_pos_manual(s_lam_q2_ii_backwards, s_num_q2_ii_backwards, ind_r)[1]
# get delta_v
delta_v_ii_backwards = delta_pos_manual(s_lam_q2_ii_backwards, s_num_q2_ii_backwards, ind_v)[1]
# get delta_a
gravitational_parameter_sun = bodies.get_body( "Sun" ).gravitational_parameter
r_lam_norm_ii_backwards, r_lam_len_ii_backwards = norm_manual(s_lam_q2_ii_backwards, ind_r)
a_lam_ii_backwards = - gravitational_parameter_sun / r_lam_len_ii_backwards**3  * r_lam_norm_ii_backwards
a_num_ii_backwards = dep_var_q2_ii_backwards[:,ind_r]
delta_a_ii_backwards = delta_pos_manual(a_lam_ii_backwards, a_num_ii_backwards, [0,1,2])[1]

# Open required data Case ii - forwards
dep_var_q2_ii_forwards = open_dat(filename = "Q2_midpoint_forwardcase_ii_dependent_variables.dat")
s_lam_q2_ii_forwards = open_dat(filename = "Q2_midpoint_forwardcase_ii_lambert_states.dat")
s_num_q2_ii_forwards = open_dat(filename = "Q2_midpoint_forwardcase_ii_numerical_states.dat")
# # set up lists of indices
ind_r = [1,2,3]
ind_v = [4,5,6]
# get time
time_vec_ii_forwards = s_num_q2_ii_forwards[:,[0]]
# get delta_r
delta_r_ii_forwards = delta_pos_manual(s_lam_q2_ii_forwards, s_num_q2_ii_forwards, ind_r)[1]
# get delta_v
delta_v_ii_forwards = delta_pos_manual(s_lam_q2_ii_forwards, s_num_q2_ii_forwards, ind_v)[1]
# get delta_a
gravitational_parameter_sun = bodies.get_body( "Sun" ).gravitational_parameter
r_lam_norm_ii_forwards, r_lam_len_ii_forwards = norm_manual(s_lam_q2_ii_forwards, ind_r)
a_lam_ii_forwards = - gravitational_parameter_sun / r_lam_len_ii_forwards**3  * r_lam_norm_ii_forwards
a_num_ii_forwards = dep_var_q2_ii_forwards[:,ind_r]
delta_a_ii_forwards = delta_pos_manual(a_lam_ii_forwards, a_num_ii_forwards, [0,1,2])[1]

case_ii_backwards_mat = np.hstack((time_vec_ii_backwards, delta_r_ii_backwards, delta_v_ii_backwards, delta_a_ii_backwards))
case_ii_forwards_mat = np.hstack((time_vec_ii_forwards, delta_r_ii_forwards, delta_v_ii_forwards, delta_a_ii_forwards))

fig_q2_b, axs_q2_b = doublecol_multiplot(case_ii_backwards_mat, case_ii_forwards_mat, initial_time = departure_epoch, log = ['True', 'True', 'True'])
savefig(fig_q2_b, 'q2_midcase')
for ii in range(3):
    print('Case %d BW, rva%d , %.1e %.3f [perc]' % (2, ii, find_peak(case_ii_backwards_mat[:,ii+1], case_ii_backwards_mat[:,0])[0], find_peak(case_ii_backwards_mat[:,ii+1], case_ii_backwards_mat[:,0])[1] / time_of_flight * constants.JULIAN_DAY * 100  ))
    print('Case %d FW, rva%d , %.1e %.3f [perc]' % (2, ii, find_peak(case_ii_forwards_mat[:,ii+1], case_ii_forwards_mat[:,0])[0], find_peak(case_ii_forwards_mat[:,ii+1], case_ii_forwards_mat[:,0])[1] / time_of_flight * constants.JULIAN_DAY * 100 ))
plt.figure()
plt.plot(norm_manual(a_num_i)[1], label = 'C1')
plt.plot(norm_manual(a_num_ii[:,])[1], label = 'C2')
plt.plot(norm_manual(a_num_iii[:,])[1], label = 'C3')
plt.legend()
plt.show()                                                    
fig_q3_a, axs_q3_a = multicol_multiplot(case_i_mat, case_ii_mat, case_iii_mat, log = [True, True, True])
savefig(fig_q3_a, 'q2_delta_rva')
# plt.show()

## Print offsets
for ii in range(3):
    print('Case %d, rva%d , %.1e %.3f [perc]' % (1, ii, find_peak(case_i_mat[:,ii+1], case_i_mat[:,0])[0], find_peak(case_i_mat[:,ii+1], case_i_mat[:,0])[1]/case_i_mat[-1,0]*100))
    print('Case %d, rva%d , %.1e %.3f [perc]' % (2, ii, find_peak(case_ii_mat[:,ii+1], case_ii_mat[:,0])[0], find_peak(case_ii_mat[:,ii+1], case_ii_mat[:,0])[1]/case_ii_mat[-1,0]*100))
    print('Case %d, rva%d , %.1e %.3f [perc]' % (3, ii, find_peak(case_iii_mat[:,ii+1], case_iii_mat[:,0])[0], find_peak(case_iii_mat[:,ii+1], case_iii_mat[:,0])[1]/case_iii_mat[-1,0]*100))

# Question 3
## 20 points; Maximum text length: 10 lines

We will now turn our attention to modifying the trajectory, so that we arrive at the desired target when *including* perturbations. When we modify an initial state $\mathbf{x}_{0}$ or some parameter vector $\mathbf{p}$ (which contains properties of the dynamical model), the propagated state $\mathbf{x}(t)$ will change. When changing the initial state by $\Delta \mathbf{x}_{0}$ and/or a parameter vector by $\Delta \mathbf{p}$, we will denote the resulting change in numerically propagated state, as a function of time, as $\Delta {\mathbf{x}}(t)$.

For given modifications $\Delta \mathbf{x}_{0}$ and $\Delta \mathbf{p}$, the behaviour of $\Delta {\mathbf{x}}(t)$ can be obtained by directly numerically repropagating $\mathbf{x}(t)$ with the modified initial state and parameter vector, and subtracting the propagation result without these modifications. However, this can be a tedious and computationally expensive process. The matrices $\boldsymbol{\Phi}(t,t_{0})$ and $\mathbf{S}(t)$ provide a way to approximate $\Delta {\mathbf{x}}(t)$, by means of a *linearization*:

\begin{align}
\boldsymbol{\Phi}(t,t_{0})=\frac{\partial\mathbf{x}(t)}{\partial\mathbf{x}_{0}}\\
 \mathbf{S}(t)=\frac{\partial\mathbf{x}(t)}{\partial\mathbf{p}}
\end{align}

Now, we denote the linearized change in state as $\Delta \tilde{\mathbf{x}}(t)$, which we compute from:
\begin{align}
\Delta \tilde{\mathbf{x}}(t)=\boldsymbol{\Phi}(t,t_{0})\Delta \mathbf{x}_{0}+\mathbf{S}(t)\Delta\mathbf{p}\label{eq:linearizedChangeInState}
\end{align}
so that $\Delta \tilde{\mathbf{x}}(t)$ is a *linear approximation* of $\Delta {\mathbf{x}}(t)$.

For this question, we split the total propagation time into 10 equisized (in time) arcs. Using the propagation settings of the previous question (case iii), the state and variational equations for the dynamics are propagated for each arc.  For each arc $i$ (with $i$ starting at 0), the initial state $\mathbf{x}_{i}(t_{0,i})$ is set so that it corresponds exactly with the Lambert targeter state  at the corresponding time $\bar{\mathbf{x}}(t_{0,i})$. This will result in a discontinuous state history over the full transfer (since the propagation is performed in an arc-wise manner). For this question, you will compute how to make the trajectory continuous in position by applying an impulsive correction maneuver at the start of each arc, such that $\mathbf{r}_{i}=\bar{\mathbf{r}}_{i}$ at the end of each arc. 

**Answer the following questions:**

**a)** Perform the arcwise propagation. Plot the deviation $\Delta r$ of the propagated state w.r.t. the Lambert arc as a function of time over the full trajectory.

**b)** Derive a model to use the results for $\boldsymbol{\Phi}_{i}(t_{i+1},t_{i})$ to compute the change in velocity $\Delta \mathbf{v}_{i}$ needed at the beginning of each arc to ensure that the spacecraft reaches $\bar{\mathbf{r}}$ at the end of the arc. Keep the initial position of the arc constant. Show the derivation of your model. For this question, neglect linearization errors (assume $\Delta \tilde{\mathbf{x}}(t)=\Delta{\mathbf{x}}(t)$)

**c)** Modify the code, so that the initial states of each arc are modified in accordance with your calculated correction manuevers in (b).  Plot the deviation $\Delta r$ of the modified propagated state w.r.t. the Lambert arc as a function of time. Do you consider your linearized model to be applicable in this specific situation? 

<!-- **d)** The model you derived in (b) will provide a trajectory that is continuous in position, but not velocity: there are velocity discontinuities between subsequent arcs. An impulsive $\Delta V$ must be provided at each of these discontinuities, as well as at the initial time  the first arc, and final time of the last arc (to ensure $\mathbf{x}=\bar{\mathbf{x}}$ at these points. Using the assumptions and derivation of (b), derive a set of equations that will provides the total $\Delta {V}$, over the full trajectory, that are required. -->



**Add to save file 1}** <br>
Row 9: initial propagation time and Cartesian state  (arc 0). <br>
Row 10: final propagation time and Cartesian state (arc 0). <br>
Row 11: initial propagation time and Cartesian state (arc 4). <br>
Row 12: final propagation time and Cartesian state (arc 4). <br>


In [ ]:
current_question = 3

# Create body objects
bodies = create_simulation_bodies( )

# Create Lambert arc state model
lambert_arc_state_model = get_lambert_problem_result(bodies, target_body, departure_epoch, arrival_epoch)

##############################################################

# Set start and end times of full trajectory
buffer_time = 2.0 * constants.JULIAN_DAY
departure_epoch_with_buffer = departure_epoch + buffer_time
arrival_epoch_with_buffer = arrival_epoch - buffer_time
travel_time = arrival_epoch_with_buffer - departure_epoch_with_buffer
# Compute number of arcs and arc length
number_of_arcs = 10
arc_length = travel_time / number_of_arcs

##############################################################

# Compute relevant parameters (dynamics, state transition matrix, Delta V) for each arc
for arc_index in range(number_of_arcs):
    
    # Compute initial and final time for arc
    current_arc_initial_time = departure_epoch_with_buffer + arc_length * arc_index
    current_arc_final_time = departure_epoch_with_buffer + arc_length * (arc_index + 1)

    ###########################################################################
    # RUN CODE FOR QUESTION 3a ################################################
    ###########################################################################
    
    # Propagate dynamics on current arc
    # STUDENT CODE TASK (call propagation function)
    dynamics_simulator = propagate_trajectory( current_arc_initial_time, current_arc_final_time, bodies, lambert_arc_state_model, 
    "Q3_arc_" + str(arc_index) + "_not_corrected", use_perturbations = True)
    state_history = dynamics_simulator.state_history
    
    # Retrieve Lambert arc for same epochs, and compute final difference
    lambert_history = get_lambert_arc_history( lambert_arc_state_model, state_history )

    # ###########################################################################
    # # RUN CODE FOR QUESTION 3d ################################################
    # ###########################################################################
    
    # # Solve for state transition matrix on current arc
    variational_equations_solver = propagate_variational_equations(
        current_arc_initial_time, current_arc_final_time, bodies, lambert_arc_state_model)  
     
    # Retrieve propagation resuls and compute Lambert arc history
    state_transition_matrix_history = variational_equations_solver.state_transition_matrix_history
    state_history = variational_equations_solver.state_history
    lambert_history = get_lambert_arc_history( lambert_arc_state_model, state_history )    
    
    final_epoch = list(lambert_history.keys())[-1]
    
    # Retrieve final state deviation between numerical and Lambert model
    final_state_deviation = state_history[ final_epoch ] - lambert_history[ final_epoch ]
    
    # Get final state transition matrix (and its inverse)
    final_epoch = list(state_transition_matrix_history.keys())[-1]
    # print('Time difference: ', final_epoch - current_arc_final_time)
    final_state_transition_matrix = state_transition_matrix_history[ final_epoch ]
    final_inverse_state_transition_matrix = np.linalg.inv( final_state_transition_matrix )

    final_state_transition_rv = final_state_transition_matrix[ :3, 3: ]
    final_inverse_transition_rv = np.linalg.inv( final_state_transition_rv )
    # Compute required velocity change at beginning of arc to meet required final state 
    initial_velocity_correction = np.dot(final_inverse_transition_rv,(-final_state_deviation[:3]))
    initial_state_correction = np.hstack(([0,0,0], initial_velocity_correction))
    # print('Initial state correction', initial_state_correction[3:])
    # Propagate with correction to initial state  
    # STUDENT CODE TASK (call propagation function). HINT: the initial state correction can be passed as an input to function
    dynamics_simulator_after_correction = propagate_trajectory( current_arc_initial_time, current_arc_final_time, bodies, lambert_arc_state_model, 
    "Q3_arc_" + str(arc_index) + "_corrected", use_perturbations = True, initial_state_correction = initial_state_correction)
    state_history_after_correction = dynamics_simulator_after_correction.state_history

    # Compute and print deviation
    final_state_deviation_after_correction = state_history_after_correction[ final_epoch ] - lambert_history[ final_epoch ]    
    print('Final position deviation after correction', final_state_deviation_after_correction[:3])
    # # Save corrected arc
    # write_propagation_results_to_file( dynamics_simulator, lambert_arc_state_model, 
    #                                  "Q3_arc_" + str(arc_index) + "_corrected", output_directory)

## Plots for Q3

In [ ]:
num_arcs = 10
print_condition = 0
# Load data files
s_lam_q3_not_corrected_raw = open_arc('Q3_arc_0_not_corrected_lambert_states.dat', num_arcs, print_condition)
s_num_q3_not_corrected_raw = open_arc('Q3_arc_0_not_corrected_numerical_states.dat', num_arcs, print_condition)
# Combine into single matrix
s_lam_q3_not_corrected = combine_arcs(s_lam_q3_not_corrected_raw)
s_num_q3_not_corrected = combine_arcs(s_num_q3_not_corrected_raw)
# Get time vector
time_vec_q3_not_corrected = s_num_q3_not_corrected[:,[0]]
# Get deltas 
ind_r = [1,2,3]
delta_r_q3_not_corrected = delta_pos_manual(s_lam_q3_not_corrected, s_num_q3_not_corrected, ind_r)[1]
# Prepare data matrix
q3_not_corrected_mat = np.hstack((time_vec_q3_not_corrected, delta_r_q3_not_corrected))
# # Plot 
fig_q3_not_corrected, ax_q3_not_corrected = delta_pos_plot(q3_not_corrected_mat, departure_epoch_with_buffer)
savefig(fig_q3_not_corrected, 'delta_r_q3_no_corr')

num_arcs = 10
print_condition = 0
# Load data files
s_lam_q3_corrected_raw = open_arc('Q3_arc_0_corrected_lambert_states.dat', num_arcs, print_condition)
s_num_q3_corrected_raw = open_arc('Q3_arc_0_corrected_numerical_states.dat', num_arcs, print_condition)
# Combine into single matrix
s_lam_q3_corrected = combine_arcs(s_lam_q3_corrected_raw)
s_num_q3_corrected = combine_arcs(s_num_q3_corrected_raw)
# Get time vector
time_vec_q3_corrected = s_num_q3_corrected[:,[0]]
# Get deltas 
ind_r = [1,2,3]
delta_r_q3_corrected = delta_pos_manual(s_lam_q3_corrected, s_num_q3_corrected, ind_r)[1]
# Prepare data matrix
q3_corrected_mat = np.hstack((time_vec_q3_corrected, delta_r_q3_corrected))
# # Plot 
fig_q3_corrected, ax_q3_corrected = delta_pos_plot(q3_corrected_mat, departure_epoch_with_buffer, labels = ['With impulsive corrections'])
savefig(fig_q3_corrected, 'delta_r_q3_corr')

# Plot single arc- corrected and not corrected
num_arcs = 1
print_condition = 0
# Load data files
s_lam_q3_corrected_raw_single = open_arc('Q3_arc_0_corrected_lambert_states.dat', num_arcs, print_condition)
s_num_q3_corrected_raw_single = open_arc('Q3_arc_0_corrected_numerical_states.dat', num_arcs, print_condition)
s_lam_q3_not_corrected_raw_single = open_arc('Q3_arc_0_not_corrected_lambert_states.dat', num_arcs, print_condition)
s_num_q3_not_corrected_raw_single = open_arc('Q3_arc_0_not_corrected_numerical_states.dat', num_arcs, print_condition)
# Combine into single matrix
s_lam_q3_corrected_single = combine_arcs(s_lam_q3_corrected_raw_single)
s_num_q3_corrected_single = combine_arcs(s_num_q3_corrected_raw_single)
s_lam_q3_not_corrected_single = combine_arcs(s_lam_q3_not_corrected_raw_single)
s_num_q3_not_corrected_single = combine_arcs(s_num_q3_not_corrected_raw_single)
# Get time vector
time_vec_q3_corrected_single = s_num_q3_corrected_single[:,[0]]
time_vec_q3_not_corrected_single = s_num_q3_not_corrected_single[:,[0]]
# Get deltas 
ind_r = [1,2,3]
delta_r_q3_corrected_single = delta_pos_manual(s_lam_q3_corrected_single, s_num_q3_corrected_single, ind_r)[1]
delta_r_q3_not_corrected_single = delta_pos_manual(s_lam_q3_not_corrected_single, s_num_q3_not_corrected_single, ind_r)[1]
# Prepare data matrix
q3_corrected_single_mat = np.hstack((time_vec_q3_corrected_single, delta_r_q3_corrected_single))
q3_not_corrected_single_mat = np.hstack((time_vec_q3_not_corrected_single, delta_r_q3_not_corrected_single))
# # Plot 
fig_q3_both_single, ax_q3_both_single = delta_pos_plot(q3_not_corrected_single_mat, departure_epoch_with_buffer, q3_corrected_single_mat)
savefig(fig_q3_both_single, 'delta_r_q3_single')
plt.show()

# Question 4
## 25 points; Maximum text length:  lines

For this question, you will perform a similar analysis as in the previous question, but now using an approximate model for a low-thrust acceleration model to correct the orbit so that it reaches the start and end of the trajectory correctly. Instead of a ten-arc model, you will use a one- and two-arc model. 

In your model, you will parameterize the thrust as a constant acceleration in RSW frame for each arc. This thrust will be added to the simulation by defining a so-called 'empirical' acceleration: a constant acceleration in RSW direction. Note that, since the direction of the RSW frame changes in time, the inertial direction of this empirical acceleration will change in time. For this question, you will compute the magnitudes of these emprical accelerations, such that the trajectory meets all the required boundary conditions.

By calculating the sensitivity matrix $\mathbf{S}$ for the entries of the empirical acceleration, you will be able to calculate (approximately) the required thust under a number of different conditions. For this question, neglect linearization errors (assume $\Delta \tilde{\mathbf{x}}(t)=\Delta{\mathbf{x}}(t)$).

**Answer the following questions:**

**a)** Consider a single arc propagation for the full transfer. Derive an equation to use the results for $\mathbf{S}(t_{E})$ to compute the low-thrust acceleration in RSW frame (denote this as $\mathbf{p}$) needed to ensure that the spacecraft reaches $\bar{\mathbf{r}}(t_{E})$ at the end of the transfer. Keep the initial state of the arc constant at $\bar{\mathbf{x}}(t_{0})$. Put no constraints on the final velocity. Show the derivation of your model, starting from the equations in the lecture videos. <br>
**b)** Implement your model for question (a) in your code, and verify and argue that it works correctly (use the same value of $\Delta t$ as in question 2, case iii). Plot any quantities you need to show the correct functioning of your model.

For the next questions, you will analyze the low-thrust trajectory for a two-arc model (with each arc having equal duration). For this question, do not impose any *a priori* constraints on the absolute position or velocity at the arc splitting point (unlike in question 3, where this point was required to correspond to $\bar{\mathbf{r}}$, but do impose a constraint of continuity in position and velocity over the full trajectory (unlike in question 3, where the velocity was discontinuous between arcs).

**c)** For an arbitrary choice of constant RSW-thrust in arc 1 (denoted $\mathbf{p}_{1}$), thrust in arc 2 (denoted $\mathbf{p}_{2}$), and modification in initial velocity of arc 1 (denoted $\mathbf{v}_{i}(t_{0,1})$), derive a single equation for the change in position at the end of arc 2 (denoted $\Delta \mathbf{r}(t_{E,2})$). Write a single explicit equation for $\Delta \mathbf{r}(t_{E,2})$, in the following notation:
\begin{align}
\Delta \mathbf{r}(t_{E,2})=\mathbf{A}\begin{pmatrix} \mathbf{p}_{1}\\ \mathbf{p}_{2} \\ \Delta\mathbf{v}_{i}(t_{0,1}) \end{pmatrix}
\end{align}
and provide an explicit formulation for the matrix $\mathbf{A}$. Use the notation from the lecture slides on *Reaching the objective - Arc-wise Low-thrust*.
<br>
**d)** The equation you derived in (c) does not have a unique solution. Choose $ \Delta\mathbf{v}_{i}(t_{0,1})=\mathbf{0}$, and set $\mathbf{p}_{1}$ equal to the value of $\mathbf{p}$ you derived and computed in questions (a) and (b). Starting from the equation you derived in (c), formulate an explicit equation for $\mathbf{p}_{2}$ needed to achieve a given $\Delta \mathbf{r}(t_{E,2})$.<br>
**e)**  Implement the model you derived in question (d) in your code, such that you obtain $\mathbf{r}(t_{E,2})=\bar{\mathbf{r}}(t_{E,2})$ (*e.g.* such that the trajectory terminates on the Lambert arc). Verify that the model works correctly: show that it meets all the required constraints.

**Add to save file 1}** <br>
Row 13: final propagation time and Cartesian state (question 4b)<br>
Row 14: final propagation time and Cartesian state (question 4e)<br>

In [ ]:
###########################################################################
# RUN CODE FOR QUESTION 4 #################################################
###########################################################################

current_question = 4
rsw_acceleration_magnitude = [0,0,0]

# Create body objects
bodies = create_simulation_bodies( )

# Create Lambert arc state model
lambert_arc_state_model = get_lambert_problem_result(bodies, target_body, departure_epoch, arrival_epoch)

###########################################################################
# RUN CODE FOR QUESTION 4b ################################################
###########################################################################

# Set start and end times of full trajectory
# STUDENT CODE TASK (fill in ...)
buffer_time = 2.0 * constants.JULIAN_DAY
departure_epoch_with_buffer = departure_epoch + buffer_time
arrival_epoch_with_buffer = arrival_epoch - buffer_time
travel_time = arrival_epoch_with_buffer - departure_epoch_with_buffer

# Solve for state transition matrix on current arc
variational_equations_solver = propagate_variational_equations(
    departure_epoch_with_buffer, arrival_epoch_with_buffer, bodies, lambert_arc_state_model)  

sensitivity_matrix_history = variational_equations_solver.sensitivity_matrix_history
state_history = variational_equations_solver.state_history
lambert_history = get_lambert_arc_history( lambert_arc_state_model, state_history )  
# Simulate without thrust corrections
dynamics_simulator_no_thrust = propagate_trajectory(  departure_epoch_with_buffer, arrival_epoch_with_buffer, bodies, lambert_arc_state_model, 
    "Q4_single_arc_no_thrust", use_perturbations = True)
# Compute low-thrust RSW acceleration to meet required final position 
# STUDENT CODE TASK: calculate low-thrust acceleration to meet arc end position requirement
# Get t_e
final_epoch = list(lambert_history.keys())[-1]
# final_epoch = arrival_epoch_with_buffer
# Get S_rp(t_e)^-1
final_sensitivity_matrix = sensitivity_matrix_history[ final_epoch ]
# final_sensitivity_matrix_rp = final_sensitivity_matrix[ :3 ] # First 3 rows correspond to the change in delta_r wrt. change in p
final_sensitivity_matrix_rp = final_sensitivity_matrix[ :3 ] # First 3 rows correspond to the change in delta_r wrt. change in p
inverted_final_sensitivity_matrix_rp = np.linalg.inv( final_sensitivity_matrix_rp )
# Get delta_r(t_e)
final_state_deviation = state_history[ final_epoch ] - lambert_history[ final_epoch ]
# final_state_deviation_single_no_thrust = final_state_deviation
final_position_deviation = final_state_deviation[:3]
print('Final position deviation, no thrust, taken from time ', (final_epoch-departure_epoch_with_buffer)/constants.JULIAN_DAY)
# Get a_rsw
# rsw_acceleration_magnitude = np.matmul(inverted_final_sensitivity_matrix_rp, - final_position_deviation)
rsw_acceleration_magnitude = np.matmul(inverted_final_sensitivity_matrix_rp, (-final_position_deviation))

# STUDENT CODE TASK (call propagation function). NOTE: Empirical acceleration with magnitude 
# rsw_acceleration_magnitude is added automatically by get_perturbed_propagator_settings function) 
dynamics_simulator_thrust = propagate_trajectory( departure_epoch_with_buffer, arrival_epoch_with_buffer, bodies, lambert_arc_state_model, 
    "Q4_single_arc_thrust", use_perturbations = True)

state_history_thrust = dynamics_simulator_thrust.state_history
lambert_history = get_lambert_arc_history( lambert_arc_state_model, state_history_thrust )    
final_epoch = list(lambert_history.keys())[-1]

final_state_deviation_thrust = state_history_thrust[ final_epoch ] - lambert_history[ final_epoch ]
print('Final state deviation with thrust', final_state_deviation_thrust)
# ###########################################################################
# # RUN CODE FOR QUESTION 4e ################################################
# ###########################################################################

# Compute number of arcs and arc length
# STUDENT CODE TASK (fill in ...)
number_of_arcs = 2
arc_length = ( arrival_epoch_with_buffer - departure_epoch_with_buffer ) / number_of_arcs

sensitivity_matrix_history_all = []
state_transition_matrix_history_all = []

## Prepare old p1
# rsw_acceleration_magnitude_old = rsw_acceleration_magnitude
# rsw_acceleration_magnitude = [0, 0, 0]
# Compute relevant parameters (dynamics, state transition matrix, Delta V) for each arc

for arc_index in range(number_of_arcs):
    
    # Compute initial and final time for arc
    # STUDENT CODE TASK (fill in ...)
    current_arc_initial_time = departure_epoch_with_buffer + arc_length * arc_index
    current_arc_final_time = departure_epoch_with_buffer + arc_length * (arc_index + 1)
   
    # STUDENT CODE TASK (run arc-wise model as defined in question (e) )
    ## Propagate variational equations
    variational_equations_solver = propagate_variational_equations(
        current_arc_initial_time, current_arc_final_time, bodies, lambert_arc_state_model)  
    
    sensitivity_matrix_history_ii = variational_equations_solver.sensitivity_matrix_history
    state_transition_matrices_ii = variational_equations_solver.state_transition_matrix_history
    # Save sensitivity matrices 
    sensitivity_matrix_history_all.append( sensitivity_matrix_history_ii )
    state_transition_matrix_history_all.append( state_transition_matrices_ii )
    
    if arc_index == 0:
        # Propagate numerically with rsw thrust
        dynamics_simulator_thrust = propagate_trajectory( current_arc_initial_time, current_arc_final_time, bodies, lambert_arc_state_model, 
            "Q4_double_arc_" + str(arc_index) + "_thrust", use_perturbations = True)    
        # Get state offset
        state_history_thrust = dynamics_simulator_thrust.state_history
        lambert_history = get_lambert_arc_history( lambert_arc_state_model, state_history_thrust )
        final_epoch = list(lambert_history.keys())[-1]
        final_state_deviation = state_history_thrust[ final_epoch ] - lambert_history[ final_epoch ]
    if arc_index == 1:
        final_epoch_arc_i = final_epoch
        ## Get initial state deviation
        # final_state_deviation_thrust
        initial_state_deviation = final_state_deviation
        # initial_state_deviation = final_state_deviation_thrust
        ## Save p1 and set p2 = 0
        rsw_acceleration_magnitude_arc_i = rsw_acceleration_magnitude
        rsw_acceleration_magnitude = [0, 0, 0]
        ## Propagate trajectory without rsw acceleration
        dynamics_simulator_no_thrust = propagate_trajectory( current_arc_initial_time, current_arc_final_time, bodies, lambert_arc_state_model, 
            "Q4_double_arc_" + str(arc_index) + '_no_thrust', use_perturbations = True, initial_state_correction = initial_state_deviation)    
        ## Get state offset of arc 2 without thrust
        state_history_no_thrust = dynamics_simulator_no_thrust.state_history
        lambert_history = get_lambert_arc_history( lambert_arc_state_model, state_history_no_thrust )
        final_epoch = list(lambert_history.keys())[-1]
        final_state_deviation_no_thrust = state_history_no_thrust[ final_epoch ] - lambert_history[ final_epoch ]
        final_position_deviation_no_thrust = final_state_deviation_no_thrust[:3]
        print('Final epochs: Arc 1 = %d days, Arc 2 = %d days' % ((final_epoch_arc_i - departure_epoch_with_buffer)/constants.JULIAN_DAY, (final_epoch - departure_epoch_with_buffer)/constants.JULIAN_DAY))
        ## Get p2
        # Take entire sensitivity and state transition matrices
        # final_position_deviation_no_thrust = final_state_deviation_no_thrust[:3]
        final_position_deviation_no_thrust = final_position_deviation
        final_sensitivity_matrix_arc_i = sensitivity_matrix_history_all[0][ final_epoch_arc_i ]
        final_sensitivity_matrix_arc_ii = sensitivity_matrix_history_all[1][ final_epoch ]
        final_state_transition_matrix_arc_ii = state_transition_matrix_history_all[1][ final_epoch ]
        # Take the required components
        final_state_transition_matrix_arc_ii_rx = final_state_transition_matrix_arc_ii[:3]
        final_sensitivity_matrix_arc_ii_rp = final_sensitivity_matrix_arc_ii[:3]
        inverse_final_sensitivity_matrix_arc_ii_rp = np.linalg.inv( final_sensitivity_matrix_arc_ii_rp )
        inner_term = -final_position_deviation_no_thrust - np.matmul(final_state_transition_matrix_arc_ii_rx,
                                                                    np.matmul(final_sensitivity_matrix_arc_i, rsw_acceleration_magnitude_arc_i)) 
        
        rsw_acceleration_magnitude = np.matmul(inverse_final_sensitivity_matrix_arc_ii_rp, inner_term)
        ## Propagate trajectory with rsw acceleration
        dynamics_simulator_thrust = propagate_trajectory( current_arc_initial_time, current_arc_final_time, bodies, lambert_arc_state_model, 
            "Q4_double_arc_" + str(arc_index) + '_thrust', use_perturbations = True, initial_state_correction = initial_state_deviation)    
        state_history_thrust = dynamics_simulator_thrust.state_history
        ## Get final state offset
        final_state_deviation_thrust = state_history_thrust[ final_epoch ] - lambert_history[ final_epoch ]
        print('Arc 2, state deviation, no thrust', final_state_deviation_no_thrust)
        print('Arc 2, state deviation, thrust', final_state_deviation_thrust)
        

## Plots

In [ ]:
## Plot - 4b - single arc thrust without velocity constraints
# Load data files
# s_lam_q4_thrust = open_dat('Q4_single_arc_thrust_lambert_states.dat')
# s_num_q4_thrust = open_dat('Q4_single_arc_thrust_numerical_states.dat')
# # Get time vector
# time_vec_q4_thrust = s_lam_q4_thrust[:,[0]]
# # Get deltas 
# ind_r = [1,2,3]
# delta_r_q4_thrust = delta_pos_manual(s_lam_q4_thrust, s_num_q4_thrust, ind_r)[1]
# # Prepare data matrix
# q4_thrust_mat = np.hstack((time_vec_q4_thrust, delta_r_q4_thrust))
# # # Plot 
# # fig_q4_thrust, ax_q4_thrust = delta_pos_plot(q4_thrust_mat, departure_epoch_with_buffer, labels = ['Single Arc with Thrust Correction'])
# # savefig(fig_q4_thrust, 'delta_r_q4_thrust')

# # Load data files
# s_lam_q4_no_thrust = open_dat('Q4_single_arc_no_thrust_lambert_states.dat')
# s_num_q4_no_thrust = open_dat('Q4_single_arc_no_thrust_numerical_states.dat')
# # Get time vector
# time_vec_q4_no_thrust = s_lam_q4_no_thrust[:,[0]]
# # Get deltas 
# ind_r = [1,2,3]
# delta_r_q4_no_thrust = delta_pos_manual(s_lam_q4_no_thrust, s_num_q4_no_thrust, ind_r)[1]
# # Prepare data matrix
# q4_no_thrust_mat = np.hstack((time_vec_q4_no_thrust, delta_r_q4_no_thrust))
# # Plot 
# fig_q4_no_thrust, ax_q4_no_thrust = delta_pos_plot(q4_no_thrust_mat, departure_epoch_with_buffer, labels = ['Single Arc without Thrust Correction'])
# savefig(fig_q4_no_thrust, 'delta_r_q4_no_thrust')

# fig_q4_both, ax_q4_both = delta_pos_plot(q4_no_thrust_mat, departure_epoch_with_buffer, q4_thrust_mat, labels = ['Single Arc without Thrust Correction', 'Single Arc with Thrust Correction'])
# savefig(fig_q4_both, 'delta_r_q4_both')

q4_no_thrust_mat = delta_data_matrix_from_arcs('Q4_single_arc_no_thrust', num_arcs = 1, print_condition = 0)
q4_thrust_mat = delta_data_matrix_from_arcs('Q4_single_arc_thrust', num_arcs = 1, print_condition = 0)
# plt.show()
q4_double_thrust = delta_data_matrix_from_arcs('Q4_double_arc_0_thrust', num_arcs = 2, print_condition = 0)
# print('No thrust mat', np.shape(q4_no_thurst_mat))
q4_double_no_thrust = delta_data_matrix_from_arcs('Q4_double_arc_1_no_thrust', num_arcs = 1, print_condition = 0)

labels_q4_all = [r'1 arc with $\mathbf{p}$', r'2 arcs with $\mathbf{p}_1 = \mathbf{p}$ and $\mathbf{p}_2 = 0$', r'2 arcs with $\mathbf{p}_1 = \mathbf{p}$ and computed $\mathbf{p}_2 = \mathbf{p}$']

fig_q4_all, ax_q4_all = delta_pos_plot_unlimited([q4_thrust_mat, q4_double_no_thrust, q4_double_thrust], departure_epoch_with_buffer, labels = labels_q4_all, log = [0,0,0])
savefig(fig_q4_all, 'q4_delta_r_all_02')

fig_q4_all, ax_q4_all = delta_vel_plot_unlimited([q4_thrust_mat, q4_double_no_thrust, q4_double_thrust], departure_epoch_with_buffer, labels = labels_q4_all, log = [0,0,0])
savefig(fig_q4_all, 'q4_delta_v_all_02')

plt.show()

# Question 5
## 25 points; Maximum text length: 20 lines

Since the model in questions 3 and 4 for computing the influence due to a small change in state and parameters is based on a linearization, it is reasonably valid for *small* values of $\Delta \mathbf{x}_{0}$ and $\mathbf{p}$ but breaks down for larger deviations. The error $\boldsymbol{\epsilon}_{\mathbf{x}}(t)$ due to the linearization can be defined as:

\begin{align}
\boldsymbol{\epsilon}_{\mathbf{x}}(t) = \Delta \tilde{\mathbf{x}}(t) -  \Delta {\mathbf{x}}(t)
\end{align}

For this question, you will numerically investigate the limit of validity of the above linearization using $\boldsymbol{\Phi}(t,t_{0})$ (the influence of $\mathbf{S}(t)$ is not considered here), for the 10-arc model of question 3.

Determine, for arc $i=0$, and arc $i=4$, independently for each of the entries of $\Delta \mathbf{x}_{0}(=\Delta \mathbf{x}(t_{0,i}))$, how large the initial state corrections are allowed to be, before the linearization used to obtain $\Delta\tilde{\mathbf{x}}(t)$ is no longer valid.

Use the following criterion as the definition of a valid linearization:
\begin{align}
\max_{t}||\boldsymbol{\epsilon}_{\mathbf{r}}(t)||<\text{100 km}\hspace{0.5cm}\vee
\hspace{0.5cm}\max_{t}||\boldsymbol{\epsilon}_{\mathbf{v}}(t)||<\text{1 m/s}
\end{align}
where $\boldsymbol{\epsilon}_{\mathbf{r}}(t)$ and $\boldsymbol{\epsilon}_{\mathbf{v}}(t)$ denote the linearization error in position and velocity (first and last three entries of $\boldsymbol{\epsilon}_{\mathbf{x}}(t)$).

Find the minimum positive value of each entry of the initial state perturbations for arcs 0 and 4, for which the error criterion is no longer true over the full arc, each time keeping the other 5 entries of $\Delta \mathbf{x}_{0}$ fixed to zero. 

Your answer must be correct to within 25% (set up your analysis so that this is guaranteed). For instance, if the true limiting value of $\Delta {y}_{0}$ is 2.4 m, any value from 1.8 m to 3.0 m will be accepted. **Implement your model in such a way that all 12 limiting values (6 for arc $i=0$; 6 for arc $i=4$) are produced from a single run of your program.**

Explain the algorithm that you have used and implemented, explaining why the model you have set up is guaranteed to give the requested results to the required accuracy. Provide *pseudo-code* (so not a copy-paste of your code!) in your explanation.


**Add to save file 2** <br>
Matrix, 6 rows by 2 columns, permitted $\Delta \mathbf{x_0}$ (arcs 0 and 4)



In [ ]:
###########################################################################
# RUN CODE FOR QUESTION 5 #################################################
###########################################################################

current_question = 5
simulation_output_directory = r'C:\Users\Beautiful\Documents\Github Repositories\NumericalAstrodynamics2020_Kipras\Assignment2\SimulationOutput'
# Create body objects
bodies = create_simulation_bodies( )

# Create Lambert arc state model
lambert_arc_state_model = get_lambert_problem_result(bodies, target_body, departure_epoch, arrival_epoch)

# Set full start and end times
buffer_time = 2.0 * constants.JULIAN_DAY
departure_epoch_with_buffer = departure_epoch + buffer_time
arrival_epoch_with_buffer = arrival_epoch - buffer_time

# Set arc length
number_of_arcs = 10
arc_length = ( arrival_epoch_with_buffer - departure_epoch_with_buffer ) / number_of_arcs

considered_arc_indices = [0, 4]

# Setup epsilon limits
eps_r_limit = 100e3 # m
eps_v_limit = 1e0 # m/s
allowed_offset = 0.001

for arc_index in considered_arc_indices:
    
    # Compute start and end time for current arc
    current_arc_initial_time = departure_epoch_with_buffer + arc_index * arc_length
    current_arc_final_time = departure_epoch_with_buffer + ( arc_index + 1 ) * arc_length

    # Get propagator settings for perturbed forward arc
    arc_initial_state = np.reshape(lambert_arc_state_model.get_cartesian_state( current_arc_initial_time ), (6,1))
    propagator_settings = get_perturbed_propagator_settings( bodies, arc_initial_state, current_arc_final_time )
    
    # Set integrator settings
    integrator_settings = propagation_setup.integrator.runge_kutta_4(
        current_arc_initial_time, fixed_step_size
    )
    
    ###########################################################################
    # PROPAGATE NOMINAL TRAJECTORY AND VARIATIONAL EQUATIONS ##################
    ###########################################################################
    
    parameters_for_which_to_compute_sensitivity = get_sensitivity_parameter_set( propagator_settings, bodies, 
                                                                                target_body)

    variational_equations_simulator2= estimation_setup.SingleArcVariationalEquationsSolver(
         bodies, integrator_settings, propagator_settings, parameters_for_which_to_compute_sensitivity, integrate_on_creation=1 )
    
    state_transition_result = variational_equations_simulator2.state_transition_matrix_history
    nominal_integration_result = variational_equations_simulator2.state_history
    
    lambert_history = get_lambert_arc_history( lambert_arc_state_model, nominal_integration_result )
        
    # TODO: Retrieve nominal initial state value (e.g. initial state with Delta x_0 = 0)
    initial_epoch = list(state_transition_result.keys())[0]
    original_initial_state = nominal_integration_result[ initial_epoch ]
    
    ###########################################################################
    # START ANALYSIS ALGORITHM FOR QUESTION 4 #################################
    ###########################################################################
    
    # This vector will hold the maximum permitted initial state perturbations for which the linearization 
    # is valid (for the current arc. The vector is initialized to 0, and each of its 6 entries is computed 
    # in the 6 iterations of the coming for loop (that runs over the iteration variable 'entry')
    permitted_perturbations = np.array([0,0,0,0,0,0])
    # Initial guesses
    initial_perturbations = np.array([100e3, 100e3, 100e3, 1, 1, 1]) # 
    # Iterate over all initial state entries
    for entry in range(6):
    # for entry in range(1):
        
        # STUDENT CODE TASK: Define (iterative) algorithm to compute current entry of 'permitted_perturbations'
        # General structure: define an initial state perturbation (perturbed_initial_state variable),
        # compute epsilon_x (see assignment), and iterate your algorithm until convergence.
        
        ## Setup initial guess
        initial_state_perturbation = np.zeros( (6,1) )

        ii = 0
        
        correction_factor = 10 # Factor used to update the guess
        convergence_check = 0 # Convergence boolean
        while ii<1000 and not convergence_check:
            
            if ii == 0: # First iteration - take initial guess
                considered_perturbation = initial_perturbations[entry]
                prev_check = [1, 1]
                # considered_perturbation = 0 # Test purposes
            else:
                prev_check = current_check
            # else: # Update guess otherwise
            ## Setup guess initial perturbation
            initial_state_perturbation[entry] = considered_perturbation
            
            # Reset propagator settings with perturbed initial state
            perturbed_initial_state = arc_initial_state + initial_state_perturbation
            propagator_settings.reset_initial_states( perturbed_initial_state )
            
            # Create simulation object and propagate dynamics with perturbed initial state
            dynamics_simulator = propagation_setup.SingleArcDynamicsSimulator(bodies, integrator_settings, propagator_settings)
            state_history_perturbed = dynamics_simulator.state_history
            
            # Retrieve state history computed directly from perturbed initial state
            integration_result = dynamics_simulator.get_equations_of_motion_numerical_solution()
            
            ## Compute epsilon_x
            epsilon_x = calculate_epsilon(nominal_integration_result, state_history_perturbed, state_transition_result, initial_state_perturbation)[0]
            
            # Check if maximum epsilon has been exceeded
            results_check = check_condition(epsilon_x)
            current_check = results_check[0]
            eps_r_max =  results_check[1] / eps_r_limit
            eps_v_max =  results_check[2] / eps_v_limit
            
            # if current_check[0]:
            #     print('Iteration %d, eps r has been exceeded and is %.2f times too large' % (ii, eps_r_max))
            # else:
            #     print('Iteration %d, eps r has not been exceeded' % ii)
            # if current_check[1]:
            #     print('Iteration %d, eps v has been exceeded and is %d times too large' % (ii, eps_v_max))
            # else:
            #     print('Iteration %d, eps v has not been exceeded' % ii)
            
            ## Check for convergence
            convergence_check = check_convergence([eps_r_max, eps_v_max], allowed_offset)
            if convergence_check:
                print('%d Found convergence for entry %d, equal to %.1e leading to eps_r=%.1e m and eps_v = %.1e m/s' % (ii, entry, considered_perturbation, results_check[1], results_check[2]))
                
            ## Determine update sign
            if eps_r_max > 1 or eps_v_max > 1:
                update_sign = -1 # Negative change in guess value
            else:
                update_sign = 1  # Positive change in guessed value 
            
            if not convergence_check: ## Update correction_factor if not converged.
                if prev_check[0] != current_check[0] or  prev_check[1] != current_check[1]:
                    correction_factor = correction_factor / 2
                    print('%d Correction factor has been udpated to %.3f' % (ii, correction_factor))
                
                ## Update guess
                if update_sign > 0:
                    considered_perturbation = considered_perturbation * (1 + correction_factor)
                else:
                    considered_perturbation = considered_perturbation / (1 + correction_factor)
            ii += 1
            
        if convergence_check:
            permitted_perturbations[entry] = considered_perturbation
    
    np.savetxt(simulation_output_directory + r"\allowed_perturbations_arc_" + str(arc_index) + '.txt', permitted_perturbations)
    print("Permitted perturbations: ", arc_index, np.transpose( permitted_perturbations ) )

# Submission and reporting instructions


**Reporting instructions - formulating equations of motion**

When asked to explicitly write out one or more accelerations

<ul>
    <li> Follow the notation from the lecture notes </li>
    <li> Use the following indices: $S$ for Sun, $E$, $V$ and $M$ for Earth, Venus and Mars, respectively, and $p$ for the spacecraft. </li>
    <li> Use a comma to separate indices as you see fit (spacecraft position w.r.t. the Sun can be written as $\mathbf{r}_{S,p}$ or $\mathbf{r}_{Sp}$). </li>
    <li> There is no need to specify the frame orientation of any of the vectors. All are assumed to be in a frame with inertial orientation. </li>
    <li> When denoting separate accelerations, always denote the body $B$ exerting, and body $A$ undergoing, the acceleration with $\mathbf{a}_{_{BA}}$. </li>
    <li> When writing out a single acceleration in terms of positions $\mathbf{r}$, always first write the total relative positions $\mathbf{r}_{_{CD}}$ as used by the acceleration model. Expand the positions further as you see fit in next steps. </li>
    <li> When writing a position $\mathbf{r}_{_{CD}}$ that is partly retrieved from the environment (if any), and  where part of the vector is numerically propagated, split the separate contributions (in a second step after the previous point). For instance, if  $\mathbf{r}_{_{ED}}$ is propagated, and $\mathbf{r}_{_{CD}}$ is used in the acceleration, write $\mathbf{r}_{_{CD}}=\mathbf{r}_{_{CE}}+\mathbf{r}_{_{ED}}$. </li>
</ul>

### Reporting instructions - change in position

When asked to plot/compute the change in total position between two simulation results $\mathbf{r}_{1}(t)$ and $\mathbf{r}_{2}(t)$. The change in total position is to be computed as $||\Delta \mathbf{r}(t)||= ||\mathbf{r}_{2}(t)-\mathbf{r}_{1}(t)||$

### Reporting instructions - figures

When using figures, take the following guidelines into account:

<ul>
    <li> Any text (legend, axis labels etc.) should be sufficiently large so as to be legible when printed on A4 paper. </li>
    <li>Each curve should be distinguishable in your plots. </li>
    <li>Adjust the scale (e.g. linear vs. logarithmic) of your plots as needed to interpret your data.</li>
    <li>Make efficient use of space for graphs and plots. Whenever possible and legible: plot multiple curves (e.g. for different runs and/or elements) in a single figure. </li>
    <li>All figures must be complete (including axis labels, legend, caption, etc.)</li>
</ul>

Points will be deducted for unreadable figures, or figures that do not clearly show information that you refer to in your discussion.

### Reporting instructions - cover page

The cover page of each report **must contain**:

<ul>
    <li> A link to the private Github repository containing the source code and output files (see below). </li> 
    <li> The names of any people with whom you cooperated (if any) </li>
    <li> The time spent per question </li>
</ul>

If any of these points are not present on your cover page, points will be deducted.


### Reporting instructions - general

Follow the provided limitations on the length of the text that you use (this excludes figures, tables and equations). Answers longer than the imposed limit will not be read beyond this limitation. Example: if the imposed limitation is 10 lines, and you write 15, we will grade the answer based **only** on the first 10 lines.

You are free to work together with your fellow students, but are required to write your own code and report. Copying/pasting from each others report/code is not accepted, and can lead to the case being referred to the Faculty Board of Examiners.

### Support instructions

In case of any questions, there are a number of options for support:

<ul>
    <li> For issues with the installation, unit tests, or the general use of Tudat, please post an issue on Github. When posting an issue, first browse through existing issues. If your problem is raised in an open issue, post in {that} issue instead of opening a new one. Note the operating system you are using. </li>
    <li> In case of questions specific to the assignments, use the Brightspace forum. As with Github, go through existing posts before opening a new one. Do no publicly post your code, or other information that provides direct answers to the questions. </li>
    <li> In-person support and Q$\&$A is also available during working lectures and open office hours. See Brightspace calendar for time and location. </li>
</ul>

See Brightspace (Course Information $\rightarrow$ Staff and Support) for details on what to prepare when asking for support.

### Submission instructions

You will not be graded on your coding style. Submission of the reports and output files is to be done through Brightspace. **Deadline for submission is January 17 2021 23:59 CET** and can also be found on Brightspace. For late submissions, 1 point (out of a total of 10) will be subtracted {per day}. So, when handing in the report $x$ days late, $\lceil{x}\rceil$ points will be deducted. *If* you have $g$ grace days left: when handing in the report $x$ days late, $\lceil{x-g}\rceil$ points will be deducted.

Submission of your final code and results files will be done through Github. Ensure that **you only commit to a private repository**. Instructions on pushing code to Github can be found in:

https://tudat-space.readthedocs.io/en/latest/_src_use_of_tools/github.html

See below for the exact files and filenames to submit:

## *Failure to comply exactly with the requirements for file contents and naming set out below will result in point deductions.*



Instructions on how to commit code to your repository is given at https://tudat-space.readthedocs.io/en/latest/_src_use_of_tools/github.html. In addition to the report, for this assignment you will  submit:


* This notebook (which can be run directly without modifications to reproduce your results). Commit and push this file to your private GitHub repository (in the $\texttt{Assignment2/}$ directory).
* A text file containing $\textbf{only}$ the initial or final time (column 1) and Cartesian states (as a row vector; columns {2-7}) from a number of your simulations, to at least 8 digits of precision. The specific simulations for which you are to save the time/state, and the row in which you are to save them are indicated in the questions. Name the file `CartesianResults_AE4868_2020_2_YYYYYYY.dat`, where YYYYYYY is your student number. Upload this file to Brightspace.
* A text file containing $\textbf{only}$ the 6x2 matrix of permitted $\Delta \mathbf{x_0}$ for question 5. Name the file `Question5_Results_AE4868_2020_2_YYYYYYY.dat`, where YYYYYYY is your student number.  Upload this file to Brightspace. 


        
## *Failure to comply exactly with the requirements for file contents and naming set out below will result in point deductions.*